In [ ]:
import os
from dotenv import load_dotenv

# load_dotenv()
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
# PREPROCESSED_INTERVIEW_DATASET = os.getenv("PREPROCESSED_INTERVIEW_DATASET")

In [ ]:
# from pinecone.grpc import PineconeGRPC as Pinecone
# from pinecone import ServerlessSpec

/Users/kieh/Library/Caches/pypoetry/virtualenvs/ai-interview-a1WfKjkB-py3.11/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## **Import Libraries / Default Setting**

In [ ]:
# pc = Pinecone(api_key=PINECONE_API_KEY)
# index_name = "v3-tech-questions"
# index = pc.Index(index_name)

In [42]:
# langchain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

from openai import OpenAI

In [ ]:
# # Vector Embedding
# from kobert_transformers import get_kobert_model, get_tokenizer
# import torch

In [ ]:
# # KoBERT 모델 및 토크나이저 로드
# model = get_kobert_model()
# tokenizer = get_tokenizer()
# client = OpenAI(api_key=OPENAI_API_KEY)

# **Functions**

In [ ]:
def generate_response(
    system_message,
    user_message,
    params=None,
    kwargs=None,
    parser_type="list",
    custom_output=None,
):
    ### 기본 모델 파라미터 설정 ###
    default_params = {
        "temperature": 0.7,
        "max_tokens": 4096,
        "top_p": 0.9,
    }
    default_kwargs = {
        "frequency_penalty": 0.5,
        "presence_penalty": 0.5,
    }

    # 사용자로부터 전달받은 params와 kwargs를 반영하여 최종 파라미터 설정
    if params is None:
        params = default_params
    else:
        params = {**default_params, **params}

    if kwargs is None:
        kwargs = default_kwargs
    else:
        kwargs = {**default_kwargs, **kwargs}

    full_params = {**params, **kwargs}

    ### 프롬프트 생성 ###
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "{system_input}"),
            ("user", "{user_input}"),
        ]
    )

    llm = ChatOpenAI(model="gpt-4o", api_key=OPENAI_API_KEY, **full_params)
    messages = prompt.format_messages(
        system_input=system_message, user_input=user_message
    )

    # 메시지 출력 함수
    def print_messages(messages):
        for message in messages:
            print(f"{message.type.capitalize()} Prompt")
            print(f"Content : {message.content}")
            print("-" * 50)

    # 메시지 출력
    # print_messages(messages)

    if parser_type == "json":
        # # print("json으로 반환")
        # # 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
        # parser = JsonOutputParser(pydantic_object=custom_output)
        # prompt = prompt.partial(format_instructions=parser.get_format_instructions())
        # ### LLM Chain 생성 및 질문 생성 ###
        # chain = prompt | llm | parser
        response = client.beta.chat.completions.parse(
            model="gpt-4o",
            **full_params,
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message},
            ],
            response_format=custom_output,
        )
        response = response.choices[0].message.content

    elif parser_type == "list":
        # 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
        chain = prompt | llm | StrOutputParser()

        response = chain.invoke(
            {"system_input": system_message, "user_input": user_message}
        )

    return response

In [ ]:
# def search_vector_db(_keyword, _sentence, _namespace):

#     # 문장을 KoBERT 모델의 입력에 맞게 토큰화
#     inputs = tokenizer(_sentence, return_tensors="pt", padding=True, truncation=True)

#     # KoBERT로 문장 임베딩 생성 (CLS 토큰의 출력을 사용)
#     with torch.no_grad():
#         outputs = model(**inputs)
#         sentence_embeddings = outputs.last_hidden_state[:, 0, :]  # CLS 토큰 사용

#     # 벡터 형태로 변환
#     searchs = sentence_embeddings.numpy()

#     # print(searchs)
#     vectors_to_search = searchs[0].tolist()

#     lower_keyword = _keyword.lower()
#     # 필터링할 조건 설정 (예: 특정 키워드나 범위 지정)
#     filter_criteria = {"tech_keyword": {"$in": [lower_keyword]}}

#     result = index.query(
#         namespace=_namespace,
#         vector=vectors_to_search,
#         top_k=3,
#         filter=filter_criteria,
#         include_values=False,
#         include_metadata=True,
#     )

#     return result

In [ ]:
# def append_semantic_search_result(_list, _keyword, original_sentence, _data):
#     matches = _data.get("matches", [])
#     for _match in matches:
#         metadata = _match.get("metadata", {})
#         tech_keyword = metadata.get("tech_keyword", "N/A")
#         searched_sentence = metadata.get("text", "N/A")
#         score = _match.get("score", "N/A")

#         # _list에 original_sentence가 없는 경우 새로운 딕셔너리 구조 생성
#         if original_sentence not in _list.setdefault(_keyword, {}):
#             _list[_keyword][original_sentence] = []

#         # 동일한 searched_sentence가 이미 있는지 확인하여 중복 방지
#         if any(
#             entry["searched_question"] == searched_sentence
#             for entry in _list[_keyword][original_sentence]
#         ):
#             continue  # 중복일 경우 추가하지 않음

#         # original_sentence의 하위 항목으로 searched_sentence와 score 추가
#         _list[_keyword][original_sentence].append(
#             {"searched_question": searched_sentence, "score": round(score, 2)}
#         )

# **Variables**

In [ ]:
# PARAMS_KNOWLEDGE_BASED = {
#     "temperature": 0.0,  # 0.0 -> 0.7
#     "max_tokens": 4096,
#     "top_p": 1.0,  # 1.0 -> 0.5
# }

# KWARGS_KNOWLEDGE_BASED = {
#     "frequency_penalty": 0.0,
#     "presence_penalty": 0.0,
# }

In [ ]:
# import json

# with open(PREPROCESSED_INTERVIEW_DATASET, "r", encoding="utf-8") as f:
#     data = json.load(f)

In [ ]:
# import pandas as pd

# # 특정 컬럼 데이터 조회
# json_uuid_list = data["uuid_list"]
# json_questions = data["questions"]
# json_embeddings = data["embeddings"]
# json_tech_fields = data["tech_fields"]
# json_tech_keywords = data["tech_keywords"]

# df_loaded = pd.DataFrame(data)

# # 변환된 DataFrame 출력
# df_loaded

,uuid_list,questions,embeddings,tech_fields,tech_keywords
0,cb784f4e-bef2-4fdb-a391-9af061ae32c0,"네이버 쇼핑 탭에서 보여지는 상품의 노출 순위를 조회할 때, 실시간성을 반영해야한다...","[0.0425320528447628, 0.2514316141605377, -0.09...",[BE],"[architecture, 실시간성]"
1,15c838a5-d712-4480-845b-d1db9680848c,크롤러가 블락되는 경우 어떻게 대응하는 것이 좋을까요?,"[-0.009540487080812454, 0.0753772035241127, -0...",[BE],[crawling]
2,c9e2adb8-303f-413b-9f42-1f21e35fdc60,"인증 방식으로 JWT를 사용해 보신 적이 있다면, JWT에 대해 간단히 설명해 주실...","[-0.15875595808029175, -0.26383936405181885, -...","[BE, FE]",[jwt]
3,e51a3ea1-9ee5-4ac8-950e-61722056375f,"JPA를 사용하실 때, N+1 문제를 겪어보셨다면 경험한 사례와 해결한 방법에 대해...","[-0.11110392212867737, -0.011802196502685547, ...",[BE],"[jpa, n+1]"
4,006e38f9-dccc-4fb4-8f3a-97fdfcb4feb0,날짜 및 상품 별 판매 데이터 집계 과정에서 발생한 성능 문제는 없었나요?,"[0.15968304872512817, -0.32694166898727417, 0....",[BE],[db]
...,...,...,...,...,...
743,a1b61fb1-4d7d-437c-a72d-2efdafd8caa2,"Model, View, Controller의 차이는 무엇인가요?","[-0.07267197221517563, 0.12576866149902344, 0....","[BE, FE]",[design pattern]
744,fff7088a-f61c-4324-acdc-ee2ae89fbef4,Forward와 Redirect의 차이는 무엇인가요?,"[-0.1148453801870346, 0.0807654932141304, 0.15...",[FE],[basic]
745,49041f30-2e89-4af5-a91e-82912da6d192,Vue.js의 라이프사이클 함수에 대해 설명해 주실 수 있나요?,"[-0.19908951222896576, -0.08817221969366074, -...",[FE],[vue.js]
746,04253a58-1d7e-49b4-b4fc-1bffae70ab4b,동기와 비동기의 차이를 예시를 들어 설명해 주실 수 있나요?,"[-0.12343694269657135, -0.15797482430934906, 0...","[BE, FE]",[basic]


In [ ]:
# from collections import Counter

# # 개수 카운팅 및 중복 없이 리스트 반환
# # 플랫 리스트 생성 및 카운팅
# flat_list = [item for sublist in json_tech_keywords for item in sublist]
# keyword_counts = Counter(flat_list)
# unique_keywords = list(keyword_counts.keys())

In [ ]:
# unique_keywords.sort()
# unique_keywords

['\x08reusability',
 'agile',
 'ajax',
 'algorithm',
 'ami',
 'angular',
 'apache',
 'api',
 'architecture',
 'aws',
 'babel',
 'basic',
 'bigdata',
 'blockchain',
 'c',
 'cache',
 'cdn',
 'ci/cd',
 'cjs',
 'closure',
 'code splitting',
 'container',
 'context api',
 'cookie',
 'cors',
 'crawling',
 'cross browsing',
 'csr',
 'css',
 'data modeling',
 'data structure',
 'data warehouse',
 'db',
 'ddd',
 'deadlock',
 'debounce',
 'design pattern',
 'design system',
 'django',
 'docker',
 'domain_finance',
 'domain_music',
 'eks',
 'elasticache',
 'emotion',
 'error',
 'es5',
 'es6',
 'eslint',
 'esm',
 'esnext',
 'event handling',
 'firewall',
 'git',
 'high availability',
 'high traffic',
 'hoisting',
 'html',
 'http',
 'https',
 'i18n',
 'ids',
 'io',
 'ips',
 'java',
 'javascript',
 'jest',
 'jpa',
 'jquery',
 'json',
 'jwt',
 'kafka',
 'kotlin',
 'kubernetes',
 'lazy loading',
 'llm',
 'load balancer',
 'local storage',
 'lock',
 'memcached',
 'message queue',
 'mobile',
 'monitorin

# **Preprocessing PF & JD**

**1. Extract Original-PF**
- without [Special Symbol, NER tags, Page numbers ...]

**2. Extract Summarized-PF**
- Contains [재직이력, 기술스택, 프로젝트]

In [15]:
tech_field = "FE"

In [16]:
pf_path = ""
PF_INPUT = """
Page 1:
[name] 이력서 프론트엔드 개발자 [email] 마케터 시절 제품이 시장에 출시 된 후, 어떻게 자리 잡는지 직접 보고 싶어 웹개발을 시작했습니다. 3개의 사내 프로덕트를 메인으로 맡아, 섬세한 비즈니스 로직을 구현하기 위해 스스로 QA 항목을 만들어 팀원들과 공유 하며 데이터를 구축해왔습니다. 이 경험을 바탕으로, 섬세한 비즈니스 흐름을 제공하는 서비스를 만드는 데 기여하고 싶습니다. 또한, 서비스가 시장에서 어떤 반응을 얻고 있는지 빠르게 파악하고 적용하고자 합니다. 최근에는 앰플리튜드, 페이스북 픽셀과 같은 데이터 분석 및 마케팅 툴을 활용하여 추천 알고리즘을 개발하고, 이를 프론 트엔드에서 어떻게 효과적으로 표현할지 고민하고 있습니다. 저는 말과 근거를 함께 제시하며 소통하는 것을 중요하게 생각합니다. 팀원들과 함께 좋은 서비스를 만들기 위해, 기술이 필요할 때는 문서와 데모를 제작하여 문제를 해결하는 방식으로 접근하고 있습니다. 기술 스택 TypeScript, React, Next.js, tailwind-css, next-auth, storybook, react-query 경력 (주)브로즈 개발팀 | SW Developer | 정규직 2024. 02. ~ 재직 중 • 사내 신규 및 리뉴얼 서비스 4건 개발 참여, Tanstack-Query, React-hook-form 사내 도입 • (FAVIEW)-공간 뷰어 랜더링 서비스 개발 • [url] • 디자인 토큰 정리 • [name] [email] [url] [url] [link] [link] [link] [link] [link] [link] [link] [link] [link] [link] [link] [link]

Page 2:
디자인 토큰 별로 파일 정리, 기존의 파생되어있던 CSS property들을 정리된 디자인 시스템으로 모두 변 경 •NaverMap API를 이용한, 자사 공간 데이터를 지도 서비스로 시각화 • CSR, React 18 환경에 맞춰, 기본 제공 로직 리팩토링 및 클러스터 내 마커(Marker)와 데이터 간 싱크 동 기화. •주기적으로 ML 서버 호출을 담당하는 비즈니스 로직 리팩토링 • 기존: 기존 로직은 일정한 호출 스케줄링이 보장이 되지않아 의도와 다르게 호출이 되었음. 의존성 배열 미스 매치로 과도한 리랜더링이 일어났음. •개선 • useRef를 이용해 동일한 간격으로 함수 호출 • Error, Done 상황에 따른 클로저 함수 작성 및 의존성 배열 재분리 및 과도한 리랜더링 방지 •React-Hook-form 도입 • 기존: Input등 인터렉션을사용하는 페이지들이 많아 상태 관리가 과도하게 늘어나 있었음. •개선 • control 메서드를 사용해 다양한 input,button,form,textarea 커스텀 컴포넌트로 분리, 코드 절반 으로 감소. • 작성 조건들을 객체로 관리하면서 각각의 페이지에서 컴포넌트를 사용할 때 일관성 유지. •이미지 최적화 • 기존:유저가 공간 대표 사진을 업로드 시, 이미지 원본 그대로 업로드해서 이미지 랜더링이 느린 이슈가 있 었음 • 개선 • 이미지 리사이징 기준 논의 후, 업로드 시 리사이즈 및 최적화 후 서버로 전송 • 사내 홈페이지 리뉴얼 • [link] • 온보딩 페이지, 이벤트 페이지를 포함한 자사 소개 사이트 개발 • 공간 뷰어 랜더링 서비스 개발 레포지토리를 해당 서비스 레포지토리로 합치는 과정에서 TS 버전 차이로 인한 타입 오류 해결 •이미지 최적화 및 속도 개선 • 로컬에서 이미지 자산의 확장자를 webp로 바꾸고, 기존 변경 전 파일 삭제 및 배럴파일의 확장자를일괄적 으로 바꿔주는 스크립트 작성 • 가장 무거운 익스텐션(파파고)이 설치된 환경에서 테스트 진행 • 라이트 하우스 기준 [link] [link] [link] [link] [link] [link] [link] [link] [link] [link] [link] [link]

Page 3:
• 성능: 37 → 73, • FCP:3.7 → 0.7~1.2 • LCP:5.2 → 2.1- 3.4 • JS절감: 약 116KiB 개선 • (STORE) - 사내 커머스 서비스 웹뷰 작업 경험 • [link] •기존의 타이포그래피 폰트는 고정값으로 설정되어있어, 작은 뷰포트 디바이스에서 기대하는 비율이 나오지 않 음. • 개선: 다양한 뷰포트에 대응하기 위해, 모바일 시안 너비에서의 px를 vw로 치환해, clamp()로 해결. • IOS,AOS 실기기 테스트를 통해 네이티브 스타일에 따른 CSS 대응 • (STORE ADMIN) - 사내 커머스 입점 점주를 위한 어드민 서비스 개발 • 현재 판매 상태를 나타내는 대시보드 페이지, 설정 페이지 개발 • 기존 사용하던, ContextAPI의 유저 action이 늘어나면서, 상태 관리의 어려움을 느낌. • Tanstack-Query 도입 후 ContextAPI, 불필요한 상태 관리 제거 • 해당 쿼리가 업데이트가 되는 시점을 직관적인 코드로 표현해 이해를 도움 밸류와이즈 개발팀 | 프론트엔드 개발자 | 프리랜서 2023. 11. ~ 2023. 11. (1개월) 프로젝트 내용 : 스톡 옵션 계산 서비스를 제공하는 프로덕트. • next-auth와 AWS cognito 통합한 로그인 구현 • amazon-cognito-identity-js 라이브러리를 사용. 클라이언트에서 로그인 요청 시, 웹 로컬 스토리지에 5개의 provider과 사용자의 정보가 저장되는 문제 확인. • 개선: next-auth의 credentials와 callback을 이용해 server-side 처리. 1개 쿠키로 정보를 관리. • 결제 심사를 위한 페이지 개발 (공지사항, FAQ, 결제 페이지) • 컴플라이언스& RFC 대응을 고려해 개발을 진행 • 서비스 운영 시, 사용자의 개인정보는 컴플레인이 많이 일어나는 요소라는 것을 알게 됨. • 개선: 팀 내 데브옵스 엔지니어 분께 개인정보 위반 여부 컨펌 받은 후, RFC 이메일 패턴, 회원 가입 시, 체크 박 스 전체 동의 조건 등 컴플라이언스를 가이드에 맞게 개발. knccapitalManagement 개발팀 | 프론트엔드 개발자 | 프리랜서 [link] [link] [link] [link] [link] [link]

Page 4:
프로젝트 내용 : 뉴질랜드 투자 스타트업 회사 소개 페이지 리뉴얼. • storybook을 이용한 소통 • UI 컴포넌트 단위 테스트 진행. • Chromatic 배포를 통해 클라이언트가 레이아웃, 반응형 테스트를 직접 확인하고 피드백을 반영할 수 있는 컨 트롤 인터페이스 제공. • 반응형 UI 구현 • 복잡한 도형들을 가진 콘텐츠를 position과 가상요소 ::after를 이용해 직접 컴포넌트 제작. • 클라이언트의 추상적인 요구와 느낌을 바탕으로 gsap을 이용해 마이크로 애니메이션 구현. • 지역 인터넷 환경을 고려한 최적화 • 현지(뉴질랜드) 네트워크 상황을 개발 시에도 고려하기 위해 slow 3G, fast 3G 환경에서 랜더링 테스트 진행. • 개선: slow 3G로 최초 진입 시, 평균 로드 시간이 3초 미만이 될 수 있도록 개발 진행. vite의 이미지 최적화를 이용.imagemin-cli 패키지를 이용해 로컬에서 한 번 더 이미지 최적화 진행. • 웹 접근성을 고려한 개발 진행 • 시멘틱 태그를 사용. • Firebase Cloud Functions를 이용한 메일 전송 기능 구현 • 기존 보유 서버가 node.js로 배포시 추가 요금 지불이 필요. • 개선: express,nodemailer를 사용해 contact form에서 받은 정보를 회사 웹 메일로 전송하는 기능을 구현. 예민한 개인정보는 Magic key로 환경 변수 처리. firebase cloud functions를 선택해 배포 진행. 프로젝트 렛플(letspl) SF34(주식회사 에스에프써티포) 2024. 01. ~ 진행 중 • 퍼블리싱, 프론트엔드 개발 진행 • 회원가입 페이지 리뉴얼 • ui 라이브러리와 종속되어있어 무분별하게 증가한 !important제거. 기존 전역 css 파일 중 중복 되거나 덮여씌여진 CSS 2000줄 이상 제거. 들여쓰기로 사용되었던 문법들을 SCSS 함수를 이용해 클래스명을 동적으로 생성하는 방식으로 코드량 감사. • 포인트 충전 페이지 UI 개발 • 튜토리얼 페이지 개발

Page 5:
• 유저 추천, 프로젝트 추천, 기능 가이드, 튜토리얼 등을 포함한 페이지 제작 포트폴리오 링크 벨로그 기술 블로그 교육 [edu] 졸업 | [edu](학사) | 스포츠마케팅학과 2017. 03. ~ 2022. 02. 대외활동 프론트엔드 스쿨 플러스 단기 심화 과정 최종 발표 심사위원 (주)멋쟁이사자처럼 2023 • 정규 프론트엔드 과정을 수료하고, 심화 과정을 수강한 학생들의 파이널 프로젝트를 심사 및 피드백. • 기획 의도에 맞는 UX 설계 수준, 완성도를 평가. • 서비스 타입의 컴플라이언스 평가. • 기본 동작 테스트, 테스트 케이스 평가. • 코드 레벨 피드백. cognito와 next-auth를 통합한 커스텀 로그인 페이지 만들기 발표 AWS 사용자 모임 - 프론트엔드 소모임 2024 • cognito-identity-js와 next-auth를 통합한 유연한 UI를 구축하는 경험을 공유. • next-auth의 credentials을 이용한 서버사이드 인증 처리 과정 소개. • 기타 자료(링크): • 발표 유튜브 영상 • 발표자료 [edu] [edu] [edu]

Page 6:
• 관련 레포지토리 링크 (주) 멋쟁이사자처럼 프론트엔드 스쿨 특강 강사 (주) 멋쟁이사자처럼 2023 1. MVP 설정을 통한 파이널 프로젝트 준비하기 (2023.08) • 특강 내용: • 2시간 특강 + 2시간 맨 투 맨 멘토링 진행. • “3주의 시간동안 어떻게 효율적으로 프로젝트를 진행 할 것인가?”라는 관점에서 학생들이 많이 겪는 방해 요소를 정리. • mvp 소개. • IA & UserFlow, 백로그 등 개발에 앞서 제작해야 하는 과정에 대한 당위성 설명. 2. 개발 프로젝트 기획 방법 (2023.02) • 특강 내용: • 경험을 녹인 기획 프로세스에 대한 전반적인 설명. • 도큐먼트 작성 방법에 대한 케이스 스터디. 3. github를 이용한 프로젝트 협업 방법 (2023.01) • 특강 내용: • 깃이슈, 마일스톤, 프로젝트 사용법 소개. • 케이스 스터디를 통한 이해 고도화. • 라이브러리의 깃 이슈와 디스커션을 활용한 정보 교환 사례 소개.
"""

In [17]:
SYSTEM_PREPROCESSING_PF_ORIGINAL = """
역할(Role):
* 당신은 주어진 txt형태의 포트폴리오를 전처리 해야합니다.

목표(Goal):
* 원본 내용을 최대한 유지하세요.
* 내용을 생략하지 마세요.

지시사항(Instructions):
* 각종 특수기호를 제거하세요.
* 각종 마스킹된 [link], [url], [name], ...등의 NER 태그들을 제거하세요.
* Page 1, Page 2, ...등 Page번호를 제거하세요
"""
pf_original = generate_response(
    SYSTEM_PREPROCESSING_PF_ORIGINAL,
    PF_INPUT,
    PARAMS_KNOWLEDGE_BASED,
    KWARGS_KNOWLEDGE_BASED,
    "list",
)

In [18]:
print(f"pf_original : {pf_original}")

pf_original : 이력서 프론트엔드 개발자 마케터 시절 제품이 시장에 출시 된 후, 어떻게 자리 잡는지 직접 보고 싶어 웹개발을 시작했습니다. 3개의 사내 프로덕트를 메인으로 맡아, 섬세한 비즈니스 로직을 구현하기 위해 스스로 QA 항목을 만들어 팀원들과 공유 하며 데이터를 구축해왔습니다. 이 경험을 바탕으로, 섬세한 비즈니스 흐름을 제공하는 서비스를 만드는 데 기여하고 싶습니다. 또한, 서비스가 시장에서 어떤 반응을 얻고 있는지 빠르게 파악하고 적용하고자 합니다. 최근에는 앰플리튜드, 페이스북 픽셀과 같은 데이터 분석 및 마케팅 툴을 활용하여 추천 알고리즘을 개발하고, 이를 프론트엔드에서 어떻게 효과적으로 표현할지 고민하고 있습니다. 저는 말과 근거를 함께 제시하며 소통하는 것을 중요하게 생각합니다. 팀원들과 함께 좋은 서비스를 만들기 위해, 기술이 필요할 때는 문서와 데모를 제작하여 문제를 해결하는 방식으로 접근하고 있습니다.

기술 스택
TypeScript, React, Next.js, tailwind-css, next-auth, storybook, react-query

경력
(주)브로즈 개발팀 | SW Developer | 정규직
2024. 02. ~ 재직 중
• 사내 신규 및 리뉴얼 서비스 4건 개발 참여, Tanstack-Query, React-hook-form 사내 도입
• (FAVIEW)-공간 뷰어 랜더링 서비스 개발
• 디자인 토큰 정리

디자인 토큰 별로 파일 정리, 기존의 파생되어있던 CSS property들을 정리된 디자인 시스템으로 모두 변경
• NaverMap API를 이용한, 자사 공간 데이터를 지도 서비스로 시각화
• CSR, React 18 환경에 맞춰, 기본 제공 로직 리팩토링 및 클러스터 내 마커(Marker)와 데이터 간 싱크 동기화.
• 주기적으로 ML 서버 호출을 담당하는 비즈니스 로직 리팩토링
• 기존: 기존 로직은 일정한 호출 스케줄링이 보장이 되지않아 의도와 다르게 호출이 되었음. 의존성 배열 미스매치로 과

In [19]:
# print(f"pf_original :\n{pf_original}")

pf_summerized는 일단 사용 X

In [20]:
# SYSTEM_PREPROCESSING_PF_SUMMERIZED ="""
# 역할(Role):
# * 당신은 주어진 txt형태의 포트폴리오를 전처리 해야합니다.

# 목표(Goal):
# * 주어진 사용자의 포트폴리오의 중요 내용을 빠뜨리지 않고 요약해야합니다.

# 지시사항(Instructions):
# * 사용자의 재직 이력, 기술 스택, 프로젝트 경험을 요약하세요.
# """

In [21]:
# pf_summerized = generate_response(SYSTEM_PREPROCESSING_PF_SUMMERIZED, PF_INPUT, PARAMS_KNOWLEDGE_BASED, KWARGS_KNOWLEDGE_BASED, "list")

In [22]:
# print(f"pf_summerized :\n{pf_summerized}")

**Extract Original PF without [Special Symbol, NER tags, Page numbers ...]**

In [23]:
jd_path = ""
JD_INPUT = """
이런 분을 찾고 있어요
DX(개발 경험)와 팀 생산성 향상을 위해 끊임없이 고민하시는 분
WEB, HTML, CSS, JavaScript, TypeScript에 대한 이해도 및 지식수준이 높으신 분
Context API, Redux, Zustand, Tanstack-Query, Graphql Client 등을 활용한 앱 상태관리에
대해 고민을 해보신 분
프론트엔드 개발 경력 2년 이하이신 분
이런 분이면 더 좋아요!
하나의 서비스에 대해 오너십을 갖고 주도적으로 운영해 보신 분
UI/UX 개선과 관련된 고민을 좋아하시는 분
몰입해서 문제를 해결하는 것을 좋아하시는 분
기술에 관심이 많고, 자기 계발을 위해 노력하시는 분
Graphql을 사용하여 개발해본 경험이 있으시고, 스키마 디자인에 대해 고민해보신 분
"""

In [24]:
SYSTEM_PREPROCESSING_JD = """
역할(Role):
* 당신은 주어진 txt형태의 채용공고를, 전처리 해야합니다.

목표(Goal):
* 기술 면접을 위한 포트폴리오 요약임을 참고하세요.

지시사항(Instructions):
* 채용 공고의 자격요건과 우대조건을 위주로 정리하세요.
* 자격요건과 우대조건은 개발자 기술 키워드를 위주로 정리하세요.

결과 설정(Output):
* 형식(Format):
    ## 자격요건:
      자격요건1,
      자격요건2,
      ...
    ## 우대사항:
      우대사항1,
      우대사항2,
      ...
"""
jd = generate_response(
    SYSTEM_PREPROCESSING_JD,
    JD_INPUT,
    PARAMS_KNOWLEDGE_BASED,
    KWARGS_KNOWLEDGE_BASED,
    "list",
)

In [25]:
print(f"jd :\n{jd}")

jd :
## 자격요건:
  WEB, HTML, CSS, JavaScript, TypeScript에 대한 높은 이해도 및 지식,
  Context API, Redux, Zustand, Tanstack-Query, Graphql Client 등을 활용한 앱 상태관리 경험,
  프론트엔드 개발 경력 2년 이하

## 우대사항:
  하나의 서비스에 대한 오너십을 갖고 주도적으로 운영한 경험,
  UI/UX 개선에 대한 관심과 경험,
  Graphql을 사용한 개발 경험 및 스키마 디자인에 대한 고민 경험,
  기술에 대한 높은 관심과 자기 계발 노력


In [ ]:
from pydantic import BaseModel, Field
from typing import List


class SubExtractKeywordFromJD(BaseModel):
    requirements_and_preferences: str = Field(description="requirements or preferences")
    is_existed_in_keywordlist: bool = Field(
        description="채용공고와 주어진 기술 키워드의 교집합이면 True, 채용공고에만 존재하면 False"
    )


class OutputExtractKeywordFromJD(BaseModel):
    tech_keywords: SubExtractKeywordFromJD = Field(
        description="채용공고에서 찾아낸 기술 키워드들"
    )

In [34]:
SYSTEM_EXTRACT_KEYWORD_FROM_JD = """
역할(Role):
* 당신은 주어진 채용공고에서 기술 키워드를 추출해야 합니다.

지시사항(Instructions):
* '사전 기술키워드'에 존재하지 않는 기술 키워드도 추가 가능합니다.
* 채용공고의 자격요건과 우대사항의 모든 기술 키워드를 포함하세요.
* 같은 의미이지만, 철자가 다른 경우 '사전 기술키워드'를 사용하세요.
* 찾아낸 기술 키워드가 '사전 기술키워드'에 포함되면 'is_existed_in_keywordlist'를 True, 그렇지 않으면 False를 반환합니다.
* requirements는 자격요건, preferences는 우대사항을 의미합니다.
* 만약 특정 기술 키워드가 자격요건과 우대사항에 모두 들어간다면, requirements_and_preferences는 requirements로 지정하세요.
* JSON 형식으로 반환하세요.

입력 예시 (Input):
* 형식(Format):
  사전 기술키워드 : {"기술 키워드"},
  채용공고 : {"채용공고"},


결과 설정(Output):
* 형식(Format):
    {
        "기술키워드 1": {
            "requirements_and_preferences": 'requirements' or 'preferences',
            "is_existed_in_keywordlist": 'True or False,'
        },
        "기술키워드 2": {
            "requirements_and_preferences": 'requirements' or 'preferences',
            "is_existed_in_keywordlist": 'True or False,'
        },

    }
"""
USER_EXTRACT_KEYWORD_FROM_JD = f"""
사전 기술키워드 : {unique_keywords},
채용공고 : {jd},
"""
### "requirements_and_preferences" - [requirements : 자격요건 / preferences : 우대사항]
### "is_existed_in_keywordlist" - [True : 기술 키워드 리스트에 존재함 / False : 기술 키워드 리스트에 존재하지 않음]
r_keywords_compared_with_keywordlist_and_jd = generate_response(
    SYSTEM_EXTRACT_KEYWORD_FROM_JD,
    USER_EXTRACT_KEYWORD_FROM_JD,
    PARAMS_KNOWLEDGE_BASED,
    KWARGS_KNOWLEDGE_BASED,
    "list",
)

cleaned_string = r_keywords_compared_with_keywordlist_and_jd.split("{", 1)[1].rsplit(
    "}", 1
)[0]
keywords_compared_with_keywordlist_and_jd = json.loads("{" + cleaned_string + "}")

In [35]:
# print(f"keywords_compared_with_keywordlist_and_jd :\n{keywords_compared_with_keywordlist_and_jd}")

# **4 Points**

1. Weak-Point : [JD - PF]
2. Check-Point : [JD & PF]
3. Wow-Point : experience based
3. Doubt-Point : logic based

## **Extract Conformity-Point (Weak-Point & Check-Point)**

### **1. 부합성(Conformity-Point : Weak-Point + Check-Point) 추출**

In [99]:
from pydantic import BaseModel, Field
from typing import List


class SubOutputExtractConformitypoint(BaseModel):
    tech_keywords: str = Field(description="기술 키워드")
    requirements_and_preferences: str = Field(description="requirements or preferences")
    is_existed_in_keywordlist: bool = Field(
        description="채용공고와 주어진 기술 키워드의 교집합이면 True, 채용공고에만 존재하면 False"
    )
    is_keywords_in_PF: bool = Field(
        description="사용자의 포트폴리오에 해당 기술 관련 내용이 있으면 True, 없으면 False"
    )
    sentences: List[str] = Field(
        description="사용자 포트폴리오에서의 기술 키워드 관련 문장"
    )


class OutputExtractConformitypoint(BaseModel):
    conformitypoint: List[SubOutputExtractConformitypoint]


SYSTEM_EXTRACT_CONFORMITYPOINT = """
역할(Role):
* 당신은 다양한 개발 분야에서 폭넓은 경험을 가진 30년차 개발자이며, 현재 기술 면접관 입니다.
* 당신은 현재 프론트엔드(FE) 및 백엔드(BE)분야의 기술 면접을 담당해야 합니다.

목표(Goal):
* 주어진 포트폴리오와 채용공고의 기술키워드를 비교해 의심스러운 포트폴리오와 채용공고의 평가항목의 부합성을 찾아내야 합니다.

맥락(Context):

지시사항(Instructions):
* 기술 키워드가 나열된 문장 (ex. python, java, typescript ...)는 기술 스펙인 경향이 높으므로 절대 추가하지 마세요. 즉 단순히 표면적으로 기술 키워드가 드러난 문장을 추가하면 안됩니다.
* 포트폴리오에 맥락이나 의미상 입력의 '채용공고의 기술 키워드'와 부합하는 부분이 존재한다면, 해당하는 모든 문장을 결과의 'sentences'에 추가하세요.
* 기술 키워드 위주로, 부합성을 판단하고 기술키워드에 존재하는 부합성 항목과 존재하지 않는 부합성 항목을 분류하세요.
* requirements_and_preferences, is_existed_in_keywordlist '채용공고의 기술 키워드'에서 입력된 값을 그대로 사용하세요.
* is_keywordsInPF는 해당 기술 키워드 관련 내용이 사용자 포트폴리오에 드러나 있다면 True, 아니라면 False를 반환하세요.
* sentences는 포트폴리오 내 해당되는 문장을 모두 반환하세요.
* 만약 {tech_keyword}가 {"tech_field"}개발자로서 기본적으로 갖춰야 하는 내용이고 포트폴리오 내 해당 키워드가 존재하지 않는다면, is_keywords_in_PF 값은 True로, sentences는 {'tech_keyword'}를 추가해주세요.

제약사항(Constraints):
* 반환 형식은 오직 JSON 형식만 반환합니다.
* 바로 파이썬 코드로 사용할 수 있게 JSON 형식에 불필요한 문자는 모두 제거하세요.
* You must respond only with JSON data without wrapping it in code blocks or any other text.

입력 예시 (Input):
* 형식(Format):
  포트폴리오 : {"포트폴리오"},
  채용공고의 기술 키워드 : {"tech_keyword"},
  분야 : {"tech_field"}

"""

USER_EXTRACT_CONFORMITYPOINT = f"""
포트폴리오 : {pf_original},
채용공고의 기술 키워드 : {keywords_compared_with_keywordlist_and_jd},
분야 : {tech_field}
"""
response = generate_response(
    SYSTEM_EXTRACT_CONFORMITYPOINT,
    USER_EXTRACT_CONFORMITYPOINT,
    PARAMS_KNOWLEDGE_BASED,
    KWARGS_KNOWLEDGE_BASED,
    "json",
    OutputExtractConformitypoint,
)

cleaned_string = response.split("{", 1)[1].rsplit("}", 1)[0]
conformitypoint = json.loads("{" + cleaned_string + "}")

print(f"conformitypoint :\n{conformitypoint}")
conformitypoint

conformitypoint :
{'conformitypoint': [{'tech_keywords': 'html', 'requirements_and_preferences': 'requirements', 'is_existed_in_keywordlist': True, 'is_keywords_in_PF': True, 'sentences': ['퍼블리싱, 프론트엔드 개발 진행', '회원가입 페이지 리뉴얼', '포인트 충전 페이지 UI 개발', '튜토리얼 페이지 개발']}, {'tech_keywords': 'css', 'requirements_and_preferences': 'requirements', 'is_existed_in_keywordlist': True, 'is_keywords_in_PF': True, 'sentences': ['디자인 토큰 정리', '디자인 토큰 별로 파일 정리, 기존의 파생되어있던 CSS property들을 정리된 디자인 시스템으로 모두 변경', 'IOS, AOS 실기기 테스트를 통해 네이티브 스타일에 따른 CSS 대응', 'ui 라이브러리와 종속되어있어 무분별하게 증가한 !important 제거. 기존 전역 css 파일 중 중복 되거나 덮여씌여진 CSS 2000줄 이상 제거.']}, {'tech_keywords': 'javascript', 'requirements_and_preferences': 'requirements', 'is_existed_in_keywordlist': True, 'is_keywords_in_PF': True, 'sentences': ['CSR, React 18 환경에 맞춰, 기본 제공 로직 리팩토링 및 클러스터 내 마커(Marker)와 데이터 간 싱크 동기화.', '주기적으로 ML 서버 호출을 담당하는 비즈니스 로직 리팩토링', 'useRef를 이용해 동일한 간격으로 함수 호출', 'Error, Done 상황에 따른 클로저 함수 작성 및 의존성 배열 재분리 및 과도한 리랜더링 방지']}, {'tech_keywords

{'conformitypoint': [{'tech_keywords': 'html',
   'requirements_and_preferences': 'requirements',
   'is_existed_in_keywordlist': True,
   'is_keywords_in_PF': True,
   'sentences': ['퍼블리싱, 프론트엔드 개발 진행',
    '회원가입 페이지 리뉴얼',
    '포인트 충전 페이지 UI 개발',
    '튜토리얼 페이지 개발']},
  {'tech_keywords': 'css',
   'requirements_and_preferences': 'requirements',
   'is_existed_in_keywordlist': True,
   'is_keywords_in_PF': True,
   'sentences': ['디자인 토큰 정리',
    '디자인 토큰 별로 파일 정리, 기존의 파생되어있던 CSS property들을 정리된 디자인 시스템으로 모두 변경',
    'IOS, AOS 실기기 테스트를 통해 네이티브 스타일에 따른 CSS 대응',
    'ui 라이브러리와 종속되어있어 무분별하게 증가한 !important 제거. 기존 전역 css 파일 중 중복 되거나 덮여씌여진 CSS 2000줄 이상 제거.']},
  {'tech_keywords': 'javascript',
   'requirements_and_preferences': 'requirements',
   'is_existed_in_keywordlist': True,
   'is_keywords_in_PF': True,
   'sentences': ['CSR, React 18 환경에 맞춰, 기본 제공 로직 리팩토링 및 클러스터 내 마커(Marker)와 데이터 간 싱크 동기화.',
    '주기적으로 ML 서버 호출을 담당하는 비즈니스 로직 리팩토링',
    'useRef를 이용해 동일한 간격으로 함수 호출',
    'Error, Done 상황에 따

In [108]:
for i in conformitypoint["conformitypoint"]:
    print(i)

{'tech_keywords': 'html', 'requirements_and_preferences': 'requirements', 'is_existed_in_keywordlist': True, 'is_keywords_in_PF': True, 'sentences': ['퍼블리싱, 프론트엔드 개발 진행', '회원가입 페이지 리뉴얼', '포인트 충전 페이지 UI 개발', '튜토리얼 페이지 개발']}
{'tech_keywords': 'css', 'requirements_and_preferences': 'requirements', 'is_existed_in_keywordlist': True, 'is_keywords_in_PF': True, 'sentences': ['디자인 토큰 정리', '디자인 토큰 별로 파일 정리, 기존의 파생되어있던 CSS property들을 정리된 디자인 시스템으로 모두 변경', 'IOS, AOS 실기기 테스트를 통해 네이티브 스타일에 따른 CSS 대응', 'ui 라이브러리와 종속되어있어 무분별하게 증가한 !important 제거. 기존 전역 css 파일 중 중복 되거나 덮여씌여진 CSS 2000줄 이상 제거.']}
{'tech_keywords': 'javascript', 'requirements_and_preferences': 'requirements', 'is_existed_in_keywordlist': True, 'is_keywords_in_PF': True, 'sentences': ['CSR, React 18 환경에 맞춰, 기본 제공 로직 리팩토링 및 클러스터 내 마커(Marker)와 데이터 간 싱크 동기화.', '주기적으로 ML 서버 호출을 담당하는 비즈니스 로직 리팩토링', 'useRef를 이용해 동일한 간격으로 함수 호출', 'Error, Done 상황에 따른 클로저 함수 작성 및 의존성 배열 재분리 및 과도한 리랜더링 방지']}
{'tech_keywords': 'typescript', 'requirements_and_prefere

#### **Extract WeakPoint & CheckPoint From ConformityPoint**

In [109]:
weak_requirements_keywords, weak_preferences_keywords = [], []
check_requirements_keywords, check_preferences_keywords = [], []
for content in conformitypoint["conformitypoint"]:
    print(content)
    if not content["is_keywords_in_PF"]:
        if content["requirements_and_preferences"] == "requirements":
            weak_requirements_keywords.append(content["tech_keywords"])
        elif content["requirements_and_preferences"] == "preferences":
            weak_preferences_keywords.append(content["tech_keywords"])
    else:
        if content["requirements_and_preferences"] == "requirements":
            check_requirements_keywords.append(content["tech_keywords"])
        elif content["requirements_and_preferences"] == "preferences":
            check_preferences_keywords.append(content["tech_keywords"])

print(f"weak_requirements_keywords : {weak_requirements_keywords}")
print(f"weak_preferences_keywords : {weak_preferences_keywords}")
print(f"check_requirements_keywords : {check_requirements_keywords}")
print(f"check_preferences_keywords : {check_preferences_keywords}")

{'tech_keywords': 'html', 'requirements_and_preferences': 'requirements', 'is_existed_in_keywordlist': True, 'is_keywords_in_PF': True, 'sentences': ['퍼블리싱, 프론트엔드 개발 진행', '회원가입 페이지 리뉴얼', '포인트 충전 페이지 UI 개발', '튜토리얼 페이지 개발']}
{'tech_keywords': 'css', 'requirements_and_preferences': 'requirements', 'is_existed_in_keywordlist': True, 'is_keywords_in_PF': True, 'sentences': ['디자인 토큰 정리', '디자인 토큰 별로 파일 정리, 기존의 파생되어있던 CSS property들을 정리된 디자인 시스템으로 모두 변경', 'IOS, AOS 실기기 테스트를 통해 네이티브 스타일에 따른 CSS 대응', 'ui 라이브러리와 종속되어있어 무분별하게 증가한 !important 제거. 기존 전역 css 파일 중 중복 되거나 덮여씌여진 CSS 2000줄 이상 제거.']}
{'tech_keywords': 'javascript', 'requirements_and_preferences': 'requirements', 'is_existed_in_keywordlist': True, 'is_keywords_in_PF': True, 'sentences': ['CSR, React 18 환경에 맞춰, 기본 제공 로직 리팩토링 및 클러스터 내 마커(Marker)와 데이터 간 싱크 동기화.', '주기적으로 ML 서버 호출을 담당하는 비즈니스 로직 리팩토링', 'useRef를 이용해 동일한 간격으로 함수 호출', 'Error, Done 상황에 따른 클로저 함수 작성 및 의존성 배열 재분리 및 과도한 리랜더링 방지']}
{'tech_keywords': 'typescript', 'requirements_and_prefere

In [94]:
# # Extract Weak-Point
# weak_point_keywords = {
#     key: value
#     for key, value in conformitypoint["conformitypoint"]
#     if not value["is_keywords_in_PF"]
# }

# # Separate the extracted keywords based on 'requirements_and_preferences' value
# weak_requirements_keywords = {
#     key: value
#     for key, value in weak_point_keywords.items()
#     if value["requirements_and_preferences"] == "requirements"
# }
# weak_preferences_keywords = {
#     key: value
#     for key, value in weak_point_keywords.items()
#     if value["requirements_and_preferences"] == "preferences"
# }

# weak_requirements_keywords, weak_preferences_keywords
# # print(f"weak_requirements_keywords : {weak_requirements_keywords}")
# # print(f"weak_preferences_keywords : {weak_preferences_keywords}")

AttributeError: 'list' object has no attribute 'items'

#### **Extract CheckPoint From ConformityPoint**

In [ ]:
# # Extract Check-Point
# check_point_keywords = {
#     key: value for key, value in conformitypoint.items() if value["is_keywords_in_PF"]
# }

# # Separate the extracted keywords based on 'requirements_and_preferences' value
# check_requirements_keywords = {
#     key: value
#     for key, value in check_point_keywords.items()
#     if value["requirements_and_preferences"] == "requirements"
# }
# check_preferences_keywords = {
#     key: value
#     for key, value in check_point_keywords.items()
#     if value["requirements_and_preferences"] == "preferences"
# }

# check_requirements_keywords, check_preferences_keywords
# # print(f"check_requirements_keywords : {check_requirements_keywords}")
# # print(f"check_preferences_keywords : {check_preferences_keywords}")

({'html': {'requirements_and_preferences': 'requirements',
   'is_existed_in_keywordlist': True,
   'is_keywords_in_PF': True,
   'sentences': ['복잡한 도형들을 가진 콘텐츠를 position과 가상요소 ::after를 이용해 직접 컴포넌트 제작.',
    '시멘틱 태그를 사용.']},
  'css': {'requirements_and_preferences': 'requirements',
   'is_existed_in_keywordlist': True,
   'is_keywords_in_PF': True,
   'sentences': ['디자인 토큰 별로 파일 정리, 기존의 파생되어있던 CSS property들을 정리된 디자인 시스템으로 모두 변경',
    'IOS, AOS 실기기 테스트를 통해 네이티브 스타일에 따른 CSS 대응',
    'ui 라이브러리와 종속되어있어 무분별하게 증가한 !important 제거. 기존 전역 css 파일 중 중복 되거나 덮여씌여진 CSS 2000줄 이상 제거.']},
  'javascript': {'requirements_and_preferences': 'requirements',
   'is_existed_in_keywordlist': True,
   'is_keywords_in_PF': True,
   'sentences': ['useRef를 이용해 동일한 간격으로 함수 호출',
    'Error, Done 상황에 따른 클로저 함수 작성 및 의존성 배열 재분리 및 과도한 리랜더링 방지']},
  'typescript': {'requirements_and_preferences': 'requirements',
   'is_existed_in_keywordlist': True,
   'is_keywords_in_PF': True,
   'sentences': ['공간 뷰어 랜더링 서비스 개발 레포지토리를 해당

### **2. Semantic Search**

In [114]:
# 자격요건
requirements_in_pf_semantic_search = {}
requirements_in_pf_wiki_search = {}
requirements_not_in_pf_semantic_search = {}
requirements_not_in_pf_wiki_search = {}

# 우대사항
preferences_in_pf_semantic_search = {}
preferences_in_pf_wiki_search = {}
preferences_not_in_pf_semantic_search = {}
preferences_not_in_pf_wiki_search = {}

for content in conformitypoint["conformitypoint"]:
    keyword = content["tech_keywords"]
    _type = content["requirements_and_preferences"]
    is_in_pf = content["is_keywords_in_PF"]
    is_in_keyword_list = content["is_existed_in_keywordlist"]
    original_sentence = content["sentences"]
    temp_searched_sentences = []

    # 자격요건
    if _type == "requirements":
        # pf에 존재 O
        if is_in_pf:
            # Semantic Search
            if is_in_keyword_list:
                for s in original_sentence:
                    searched_data = search_vector_db(keyword, s, tech_field)
                    append_semantic_search_result(
                        requirements_in_pf_semantic_search, keyword, s, searched_data
                    )
            # Wiki Search
            elif not is_in_keyword_list:
                pass
        # pf에 존재 X
        elif not is_in_pf:
            # Semantic Search
            if is_in_keyword_list:
                searched_data = search_vector_db(keyword, keyword, tech_field)
                append_semantic_search_result(
                    requirements_not_in_pf_semantic_search,
                    keyword,
                    keyword,
                    searched_data,
                )
            # Wiki Search
            elif not is_in_keyword_list:
                pass

    # 우대사항
    elif _type == "preferences":
        # pf에 존재 O
        if is_in_pf:
            # Semantic Search
            if is_in_keyword_list:
                for s in original_sentence:
                    searched_data = search_vector_db(keyword, s, tech_field)
                    append_semantic_search_result(
                        preferences_in_pf_semantic_search, keyword, s, searched_data
                    )
            # Wiki Search
            elif not is_in_keyword_list:
                pass
        # pf에 존재 X
        elif not is_in_pf:
            # Semantic Search
            if is_in_keyword_list:
                searched_data = search_vector_db(keyword, keyword, tech_field)
                append_semantic_search_result(
                    preferences_not_in_pf_semantic_search,
                    keyword,
                    keyword,
                    searched_data,
                )
            # Wiki Search
            elif not is_in_keyword_list:
                pass

In [115]:
print(f"requirements_in_pf_semantic_search : {requirements_in_pf_semantic_search}")
print(f"requirements_in_pf_wiki_search : {requirements_in_pf_wiki_search}")
print(
    f"requirements_not_in_pf_semantic_search : {requirements_not_in_pf_semantic_search}"
)
print(f"requirements_not_in_pf_wiki_search : {requirements_not_in_pf_wiki_search}")
print(f"preferences_in_pf_semantic_search : {preferences_in_pf_semantic_search}")
print(f"preferences_in_pf_wiki_search : {preferences_in_pf_wiki_search}")
print(
    f"preferences_not_in_pf_semantic_search : {preferences_not_in_pf_semantic_search}"
)
print(f"preferences_not_in_pf_wiki_search : {preferences_not_in_pf_wiki_search}")

requirements_in_pf_semantic_search : {'html': {'퍼블리싱, 프론트엔드 개발 진행': [{'searched_question': '프론트엔드 개발 시, React가 아닌 HTML, CSS, 바닐라 자바스크립트로만 프로젝트를 만들어본 경험이 있으신가요?', 'score': 0.61}, {'searched_question': 'HTML5에서 도입된 주요 태그들에 대해 설명해 주세요.', 'score': 0.6}, {'searched_question': '<section>과 <article> 태그의 차이에 대해 설명해 주세요.', 'score': 0.55}], '회원가입 페이지 리뉴얼': [{'searched_question': '프로그레시브 렌더링(Progressive Rendering)이 무엇인지 설명해 주세요.', 'score': 0.55}, {'searched_question': '시맨틱 태그(Semantic Tag)가 무엇인지 설명해 주세요.', 'score': 0.54}, {'searched_question': 'HTML5에서 도입된 주요 태그들에 대해 설명해 주세요.', 'score': 0.53}], '포인트 충전 페이지 UI 개발': [{'searched_question': 'HTML5에서 도입된 주요 태그들에 대해 설명해 주세요.', 'score': 0.68}, {'searched_question': '프로그레시브 렌더링(Progressive Rendering)이 무엇인지 설명해 주세요.', 'score': 0.66}, {'searched_question': 'HTML5와 기존 HTML의 차이점을 설명해주세요.', 'score': 0.66}], '튜토리얼 페이지 개발': [{'searched_question': 'HTML5에서 도입된 주요 태그들에 대해 설명해 주세요.', 'score': 0.59}, {'searched_question': '프론트엔드 개발 시, React가 아닌 HTML, CSS, 바닐라 자바스크립

### **Weak-Point**

#### **대질문 예시 [JD-PF/차집합]**

In [139]:
SYSTEM_GENERATE_MAIN_QUESTIONS_WEAKPOINT = """
역할(Role):
* 당신은  다양한 개발 분야에서 폭넓은 경험을 가진 30년차 개발자이며 10년차 IT교육자 김철수입니다.
* 당신은 현재 프론트엔드(FE) 및 백엔드(BE)분야의 기술 역량평가를 진행하는 평가 담당관입니다.
* 역량평가 및 면접을 위해 적절한 대질문을 생성해야합니다.

목표(Goal):
* 역량평가 및 면접을 위해 적절한 질문을 생성해야합니다.

지시사항(Instructions):
* 각 키워드별로 면접 질문을 키워드마다 각각 생성하세요.
  1. 질문 의도는 이 사용자가 해당 기술을 사용해 봤는지에 대한 사실 확인 목적입니다.
  2. 질문 유형은 경험적인 질문으로 생성하세요.
  3. 다음과 같은 Format을 참고하세요. "채용공고에서 {tech_keyword}를 요구하고 있는데, 포트폴리오/이력서에는 해당 내용이 존재하지 않습니다. 혹시 {tech_keyword}를 사용해보신 경험이나 이론적으로 알고 계신 내용이 있나요?"
* 출력의 {keyword}는 반드시 입력의 '기술키워드 : {"tech_keywords"}'만 사용하세요.
* 출력의 example은 면접에 대한 사용자의 응답 가이드입니다. 이는 사용자 답변의 평가 기준으로 사용할 예정입니다.
제약사항(Constraints):
* JSON 형식만 반환합니다.
* You must respond only with JSON data without wrapping it in code blocks or any other text.


입력 예시 (Input):
* 형식(Format):
  기술키워드 : {"tech_keywords"}
"""

##### **Weak-Point / 대질문 / requirements**
변수명 : questions_requirements_not_in_pf

In [ ]:
class SubOutputGenerateMainQuestionsWeakpointRequirements(BaseModel):
    question: str = Field(description="생성된 질문")
    tech_keyword: List[str] = Field(description="질문에 해당하는 기술 키워드들")
    question_type: str = Field(description="질문 유형 : 경험")
    purpose: str = Field(description="질문 의도")
    example: List[str] = Field(
        description="우수 : '우수 답변 예시', 보통 : '보통 답변 예시', 미흡 : '미흡 답변 예시'"
    )
    reason: str = Field(description="질문 의도")


class OutputGenerateMainQuestionsWeakpointRequirements(BaseModel):
    requirements: List[SubOutputGenerateMainQuestionsWeakpointRequirements]


USER_GENERATE_MAIN_QUESTIONS_WEAKPOINT_REQUIREMENTS = f"""
기술키워드 : {weak_requirements_keywords}
"""
r_weakpoint_requirements = ""
if weak_requirements_keywords:
    r_weakpoint_requirements = generate_response(
        SYSTEM_GENERATE_MAIN_QUESTIONS_WEAKPOINT,
        USER_GENERATE_MAIN_QUESTIONS_WEAKPOINT_REQUIREMENTS,
        PARAMS_KNOWLEDGE_BASED,
        KWARGS_KNOWLEDGE_BASED,
        "json",
        OutputGenerateMainQuestionsWeakpointRequirements,
    )
cleaned_string = r_weakpoint_requirements.split("{", 1)[1].rsplit("}", 1)[0]
main_questions_weakpoint_requirements = json.loads("{" + cleaned_string + "}")

In [155]:
main_questions_weakpoint_requirements

{'requirements': [{'question': '채용공고에서 redux를 요구하고 있는데, 포트폴리오/이력서에는 해당 내용이 존재하지 않습니다. 혹시 redux를 사용해보신 경험이나 이론적으로 알고 계신 내용이 있나요?',
   'tech_keyword': ['redux'],
   'question_type': '경험',
   'purpose': '이 사용자가 해당 기술을 사용해 봤는지에 대한 사실 확인',
   'example': ["우수 : '네, Redux를 사용하여 상태 관리를 구현한 경험이 있습니다. 특히 대규모 애플리케이션에서 상태를 중앙에서 관리하는 데 유용했습니다.'",
    "보통 : 'Redux에 대해 공부한 적은 있지만 실제 프로젝트에 적용해본 적은 없습니다.'",
    "미흡 : 'Redux가 무엇인지 잘 모릅니다.'"],
   'reason': 'Redux 사용 경험을 통해 상태 관리에 대한 이해도를 평가하기 위함입니다.'},
  {'question': '채용공고에서 zustand를 요구하고 있는데, 포트폴리오/이력서에는 해당 내용이 존재하지 않습니다. 혹시 zustand를 사용해보신 경험이나 이론적으로 알고 계신 내용이 있나요?',
   'tech_keyword': ['zustand'],
   'question_type': '경험',
   'purpose': '이 사용자가 해당 기술을 사용해 봤는지에 대한 사실 확인',
   'example': ["우수 : '네, zustand를 사용하여 React 애플리케이션의 상태 관리를 간단하게 구현한 경험이 있습니다. 특히 경량 상태 관리가 필요할 때 유용했습니다.'",
    "보통 : 'zustand에 대해 들어본 적은 있지만, 실제로 사용해본 적은 없습니다.'",
    "미흡 : 'zustand가 무엇인지 잘 모릅니다.'"],
   'reason': 'Zustand 사용 경험을 통해 경량 상태 관리 라이브러리에 대한 이해도를 평가하기 위함입니다.'}]}

##### **Weak-Point / 대질문 / preferences**
변수명 : questions_requirements_not_in_pf

In [ ]:
class SubOutputGenerateMainQuestionsWeakpointPreferences(BaseModel):
    question: str = Field(description="생성된 질문")
    tech_keyword: List[str] = Field(description="질문에 해당하는 기술 키워드들")
    question_type: str = Field(description="질문 유형 : 경험")
    purpose: str = Field(description="질문 의도")
    example: List[str] = Field(
        description="우수 : '우수 답변 예시', 보통 : '보통 답변 예시', 미흡 : '미흡 답변 예시'"
    )
    reason: str = Field(description="질문 의도")


class OutputGenerateMainQuestionsWeakpointPreferences(BaseModel):
    preferences: List[SubOutputGenerateMainQuestionsWeakpointPreferences]


USER_GENERATE_MAIN_QUESTIONS_WEAKPOINT_PREFERENCES = f"""
기술키워드 : {weak_preferences_keywords},
"""
r_weakpoint_preferences = ""
if weak_preferences_keywords:
    r_weakpoint_preferences = generate_response(
        SYSTEM_GENERATE_MAIN_QUESTIONS_WEAKPOINT,
        USER_GENERATE_MAIN_QUESTIONS_WEAKPOINT_PREFERENCES,
        PARAMS_KNOWLEDGE_BASED,
        KWARGS_KNOWLEDGE_BASED,
        "json",
        OutputGenerateMainQuestionsWeakpointPreferences,
    )
cleaned_string = r_weakpoint_preferences.split("{", 1)[1].rsplit("}", 1)[0]
main_questions_weakpoint_preferences = json.loads("{" + cleaned_string + "}")

In [157]:
main_questions_weakpoint_preferences

{'preferences': [{'question': '채용공고에서 graphql를 요구하고 있는데, 포트폴리오/이력서에는 해당 내용이 존재하지 않습니다. 혹시 graphql를 사용해보신 경험이나 이론적으로 알고 계신 내용이 있나요?',
   'tech_keyword': ['graphql'],
   'question_type': '경험',
   'purpose': '이 사용자가 해당 기술을 사용해 봤는지에 대한 사실 확인',
   'example': ["우수 : '네, 이전 프로젝트에서 GraphQL을 사용하여 API를 구축한 경험이 있습니다. 특히 데이터 페칭 최적화에 많은 도움을 받았습니다.'",
    "보통 : 'GraphQL에 대해 공부한 적은 있지만, 실제 프로젝트에 적용해본 경험은 없습니다.'",
    "미흡 : 'GraphQL이 무엇인지 잘 모릅니다.'"],
   'reason': 'GraphQL을 사용한 경험이 있는지 확인하여, 해당 기술에 대한 실무 능력을 평가하기 위함입니다.'}]}

In [158]:
main_questions_weakpoint = (
    main_questions_weakpoint_requirements | main_questions_weakpoint_preferences
)
main_questions_weakpoint

{'requirements': [{'question': '채용공고에서 redux를 요구하고 있는데, 포트폴리오/이력서에는 해당 내용이 존재하지 않습니다. 혹시 redux를 사용해보신 경험이나 이론적으로 알고 계신 내용이 있나요?',
   'tech_keyword': ['redux'],
   'question_type': '경험',
   'purpose': '이 사용자가 해당 기술을 사용해 봤는지에 대한 사실 확인',
   'example': ["우수 : '네, Redux를 사용하여 상태 관리를 구현한 경험이 있습니다. 특히 대규모 애플리케이션에서 상태를 중앙에서 관리하는 데 유용했습니다.'",
    "보통 : 'Redux에 대해 공부한 적은 있지만 실제 프로젝트에 적용해본 적은 없습니다.'",
    "미흡 : 'Redux가 무엇인지 잘 모릅니다.'"],
   'reason': 'Redux 사용 경험을 통해 상태 관리에 대한 이해도를 평가하기 위함입니다.'},
  {'question': '채용공고에서 zustand를 요구하고 있는데, 포트폴리오/이력서에는 해당 내용이 존재하지 않습니다. 혹시 zustand를 사용해보신 경험이나 이론적으로 알고 계신 내용이 있나요?',
   'tech_keyword': ['zustand'],
   'question_type': '경험',
   'purpose': '이 사용자가 해당 기술을 사용해 봤는지에 대한 사실 확인',
   'example': ["우수 : '네, zustand를 사용하여 React 애플리케이션의 상태 관리를 간단하게 구현한 경험이 있습니다. 특히 경량 상태 관리가 필요할 때 유용했습니다.'",
    "보통 : 'zustand에 대해 들어본 적은 있지만, 실제로 사용해본 적은 없습니다.'",
    "미흡 : 'zustand가 무엇인지 잘 모릅니다.'"],
   'reason': 'Zustand 사용 경험을 통해 경량 상태 관리 라이브러리에 대한 이해도를 평가하기 위함입니다.'}],
 'preference

#### **꼬리질문 예시 [JD-PF/차집합]**

In [168]:
SYSTEM_GENERATE_FOLLOWED_QUESTIONS_WEAKPOINT = """
역할(Role):
* 당신은  다양한 개발 분야에서 폭넓은 경험을 가진 30년차 개발자이며 10년차 IT교육자 김철수입니다.
* 당신은 현재 프론트엔드(FE) 및 백엔드(BE)분야의 기술 역량평가를 진행하는 평가 담당관입니다.
* 역량평가 및 면접을 위해 적절한 질문을 생성해야합니다.

목표(Goal):
* 역량평가 및 면접을 위해 적절한 질문을 생성해야합니다.

* 벡터 DB내 Similarity Search를 통한 결과물인 실제 면접 질문을 참고하여 적절한 면접 질문을 생성합니다.

지시사항(Instructions):
* 각 키워드별로 면접 질문을 키워드마다 각각 3개 이상 생성하세요.
* 검색된 질문이 해당 기술키워드와 관련이 없다면 변형해서 관련있는 질문을 생성하세요.
* 출력의 {tech_keyword}는 Result of Similarity Search[tech_keyword]를 그대로 사용합니다.

제약사항(Constraints):
* JSON 형식만 반환합니다.
* You must respond only with JSON data without wrapping it in code blocks or any other text.


입력 예시 (Input):
* 형식(Format):
  Result of Similarity Search:
  {
  '기술키워드1': [
    {'original_sentence': '원본 질문 1',
      'searched_question': '원본 질문 1에 대하여 검색된 질문 1',
      'score': '유사도 점수'},
    {'original_sentence': '원본 질문 2',
      'searched_question': '원본 질문 2에 대하여 검색된 질문 2',
      'score': '유사도 점수'},
      ...
    ],
  '기술키워드2': [
    {'original_sentence': '원본 질문 1',
      'searched_question': '원본 질문 1에 대하여 검색된 질문 1',
      'score': '유사도 점수'},
    {'original_sentence': '원본 질문 2',
      'searched_question': '원본 질문 2에 대하여 검색된 질문 2',
      'score': '유사도 점수'},
      ...
    ]
  }

"""

##### **Weak-Point / 꼬리질문 / requirements**
변수명 : questions_requirements_not_in_pf_semantic_search

In [ ]:
class SubOutputGenerateFollowedQuestionsRequirements(BaseModel):
    tech_keyword: str = Field(description="질문에 해당하는 기술 키워드")
    questions: List[str] = Field(description="생성된 꼬리 질문들")


class OutputGenerateFollowedQuestionsWeakpointRequirements(BaseModel):
    requirements: List[SubOutputGenerateFollowedQuestionsRequirements]


USER_GENERATE_FOLLOWED_QUESTIONS_WEAKPOINT_REQUIREMENTS = f"""
Result of Similarity Search: {requirements_not_in_pf_semantic_search}
"""
followed_questions_weakpoint_requirements = {}
if requirements_not_in_pf_semantic_search:
    r_followed_questions_weakpoint_requirements = generate_response(
        SYSTEM_GENERATE_FOLLOWED_QUESTIONS_WEAKPOINT,
        USER_GENERATE_FOLLOWED_QUESTIONS_WEAKPOINT_REQUIREMENTS,
        PARAMS_KNOWLEDGE_BASED,
        KWARGS_KNOWLEDGE_BASED,
        "json",
        OutputGenerateFollowedQuestionsWeakpointRequirements,
    )
    cleaned_string = r_followed_questions_weakpoint_requirements.split("{", 1)[
        1
    ].rsplit("}", 1)[0]
    followed_questions_weakpoint_requirements = json.loads("{" + cleaned_string + "}")
else:
    followed_questions_weakpoint_requirements["requirements"] = {
        "tech_keyword": None,
        "questions": None,
    }

In [180]:
followed_questions_weakpoint_requirements

{'requirements': [{'tech_keyword': 'redux',
   'questions': ['Redux를 사용하여 상태 관리를 구현할 때의 주요 이점은 무엇인가요?',
    'Redux의 미들웨어를 활용한 비동기 작업 처리는 어떻게 이루어지나요?',
    'Redux의 액션과 리듀서의 역할과 관계에 대해 설명해 주세요.']},
  {'tech_keyword': 'zustand',
   'questions': ['Zustand를 사용하여 상태 관리를 구현할 때의 장점은 무엇인가요?',
    'Zustand의 상태 관리 방식과 Redux의 차이점은 무엇인가요?',
    'Zustand를 사용한 프로젝트에서의 경험을 공유해 주세요.']}]}

##### **Weak-Point / 꼬리질문 / preferences**
변수명 : questions_preferences_not_in_pf_semantic_search

In [210]:
class SubOutputGenerateFollowedQuestionsWeakpointPreferences(BaseModel):
    tech_keyword: str = Field(description="질문에 해당하는 기술 키워드")
    questions: List[str] = Field(description="생성된 꼬리 질문들")


class OutputGenerateFollowedQuestionsWeakpointPreferences(BaseModel):
    preferences: List[SubOutputGenerateFollowedQuestionsWeakpointPreferences]


USER_GENERATE_FOLLOWED_QUESTIONS_WEAKPOINT_PREFERENCES = f"""
Result of Similarity Search: {preferences_not_in_pf_semantic_search}
"""
followed_questions_weakpoint_preferences = {}
if preferences_not_in_pf_semantic_search:
    r_followed_questions_weakpoint_preferences = generate_response(
        SYSTEM_GENERATE_FOLLOWED_QUESTIONS_WEAKPOINT,
        USER_GENERATE_FOLLOWED_QUESTIONS_WEAKPOINT_PREFERENCES,
        PARAMS_KNOWLEDGE_BASED,
        KWARGS_KNOWLEDGE_BASED,
        "json",
        OutputGenerateFollowedQuestionsWeakpointPreferences,
    )
    cleaned_string = r_followed_questions_weakpoint_preferences.split("{", 1)[1].rsplit(
        "}", 1
    )[0]
    followed_questions_weakpoint_preferences = json.loads("{" + cleaned_string + "}")
else:
    followed_questions_weakpoint_preferences["preferences"] = {
        "tech_keyword": None,
        "questions": None,
    }

In [211]:
followed_questions_weakpoint_preferences

{'preferences': {'tech_keyword': None, 'questions': None}}

In [185]:
followed_questions_weakpoint = (
    followed_questions_weakpoint_requirements | followed_questions_weakpoint_preferences
)
followed_questions_weakpoint

{'requirements': [{'tech_keyword': 'redux',
   'questions': ['Redux를 사용하여 상태 관리를 구현할 때의 주요 이점은 무엇인가요?',
    'Redux의 미들웨어를 활용한 비동기 작업 처리는 어떻게 이루어지나요?',
    'Redux의 액션과 리듀서의 역할과 관계에 대해 설명해 주세요.']},
  {'tech_keyword': 'zustand',
   'questions': ['Zustand를 사용하여 상태 관리를 구현할 때의 장점은 무엇인가요?',
    'Zustand의 상태 관리 방식과 Redux의 차이점은 무엇인가요?',
    'Zustand를 사용한 프로젝트에서의 경험을 공유해 주세요.']}],
 'preferences': {'tech_keyword': None, 'questions': None}}

### **Check-Point**

#### **대질문 예시 [JD&PF/교집합]**

In [ ]:
# 원하는 데이터 구조를 정의합니다.
class SubOutputGenerateMainQuestionsCheckpointJDandPF(BaseModel):
    question: str = Field(description="생성된 질문")
    tech_keyword: List[str] = Field(description="질문에 해당하는 기술 키워드들")
    question_type: str = Field(description="질문 유형 : 경험")
    purpose: str = Field(description="질문 의도")
    example: List[str] = Field(
        description="우수 : '우수 답변 예시', 보통 : '보통 답변 예시', 미흡 : '미흡 답변 예시'"
    )
    reason: str = Field(description="질문 의도")


class OutputGenerateMainQuestionsCheckpointJDandPF(BaseModel):
    requirements: List[SubOutputGenerateMainQuestionsCheckpointJDandPF]


SYSTEM_GENERATE_MAIN_QUESTIONS_CHECKPOINT = """

역할(Role):
* 당신은  다양한 개발 분야에서 폭넓은 경험을 가진 30년차 개발자이며 10년차 IT교육자 김철수입니다.
* 당신은 현재 프론트엔드(FE) 및 백엔드(BE)분야의 기술 역량평가를 진행하는 평가 담당관입니다.
* 역량평가 및 면접을 위해 적절한 대질문을 생성해야합니다.

목표(Goal):
* 역량평가 및 면접을 위해 적절한 질문을 생성해야합니다.

지시사항(Instructions):
* 각 키워드별로 면접 질문을 키워드마다 각각 생성하세요.
  1. 질문 의도는 이 사용자가 해당 기술을 사용해 봤는지에 대한 사실 확인 목적입니다.
  2. 질문 유형은 경험적인 질문으로 생성하세요.

* 출력의 {tech_keyword}는 반드시 입력의 'keywords : {"기술키워드"}'만 사용하세요.
* 출력의 example은 면접에 대한 사용자의 응답 가이드입니다. 이는 사용자 답변의 평가 기준으로 사용할 예정입니다.
* 입력된 'pf_original'를 통해 생성된 질문이 답변 가능하지 않은 질문만 생성하세요.
* 가능하다면, 프로젝트명을 언급하세요.

제약사항(Constraints):
* JSON 형식만 반환합니다.
* You must respond only with JSON data without wrapping it in code blocks or any other text.


입력 예시 (Input):
* 형식(Format):
  포트폴리오: {pf_original},
  keywords:{
            '기술키워드1': {'requirements_and_preferences': 'requirements',
                          'is_existed_in_keywordlist': True/False,
                          'is_keywords_in_PF': True,
                          'sentences': ['포트폴리오 내 문장1',
                                        '포트폴리오 내 문장2',
                                        ...]},
            '기술키워드2': {'requirements_and_preferences': 'requirements',
                          'is_existed_in_keywordlist': True/False,
                          'is_keywords_in_PF': True,
                          'sentences': ['포트폴리오 내 문장1',
                                        '포트폴리오 내 문장2',
                                        ...]},
            ...
            }

결과 설정(Output):
* 형식(Format) :
    "1": {
      question : '생성된 질문',
      tech_keyword : '질문에 해당하는 기술 키워드들',
      question_type : '질문 종류 : 경험',
      purpose: str : '질문 의도',
      example: [우수 : '우수 답변 예시', 보통 : '보통 답변 예시', 미흡 : '미흡 답변 예시'],
      reason : '질문 생성 이유'
    },
    "2": {
      question : '생성된 질문',
      tech_keyword : '질문에 해당하는 기술 키워드들',
      question_type : '질문 종류 : 경험',
      purpose: str : '질문 의도',
      example: [우수 : '우수 답변 예시', 보통 : '보통 답변 예시', 미흡 : '미흡 답변 예시'],
      reason : '질문 생성 이유'
    },
    ...
"""

##### **Check-Point / 대질문 / requirements**
변수명 : questions_requirements_in_pf

In [ ]:
USER_GENERATE_MAIN_QUESTIONS_CHECKPOINT_REQUIREMENTS = f"""
포트폴리오: {pf_original},
keywords: {check_requirements_keywords}
"""

if check_requirements_keywords:
    r_main_questions_checkpoint_requirements = generate_response(
        SYSTEM_GENERATE_MAIN_QUESTIONS_CHECKPOINT,
        USER_GENERATE_MAIN_QUESTIONS_CHECKPOINT_REQUIREMENTS,
        PARAMS_KNOWLEDGE_BASED,
        KWARGS_KNOWLEDGE_BASED,
        "json",
        OutputGenerateMainQuestionsCheckpointJDandPF,
    )
    cleaned_string = r_main_questions_checkpoint_requirements.split("{", 1)[1].rsplit(
        "}", 1
    )[0]
    main_questions_checkpoint_requirements = json.loads("{" + cleaned_string + "}")

##### **Check-Point / 대질문 / preferences**
변수명 : questions_preferences_in_pf

In [ ]:
USER_GENERATE_MAIN_QUESTIONS_CHECKPOINT_PREFERENCES = f"""
포트폴리오: {pf_original},
keywords: {check_requirements_keywords}
"""

if check_preferences_keywords:
    r_main_questions_checkpoint_preferences = generate_response(
        SYSTEM_GENERATE_MAIN_QUESTIONS_CHECKPOINT,
        USER_GENERATE_MAIN_QUESTIONS_CHECKPOINT_PREFERENCES,
        PARAMS_KNOWLEDGE_BASED,
        KWARGS_KNOWLEDGE_BASED,
        "json",
        OutputGenerateMainQuestionsCheckpointJDandPF,
    )
    cleaned_string = r_main_questions_checkpoint_preferences.split("{", 1)[1].rsplit(
        "}", 1
    )[0]
    main_questions_checkpoint_preferences = json.loads("{" + cleaned_string + "}")

In [197]:
main_questions_checkpoint_preferences = (
    main_questions_checkpoint_requirements | main_questions_checkpoint_preferences
)
main_questions_checkpoint_preferences

{'requirements': [{'question': 'TypeScript를 사용하여 프로젝트에서 어떤 문제를 해결했는지 설명해 주실 수 있나요? 예를 들어, 특정 프로젝트에서 TypeScript의 타입 시스템을 활용하여 발생할 수 있는 오류를 어떻게 예방했는지 궁금합니다.',
   'tech_keyword': ['typescript'],
   'question_type': '경험',
   'purpose': '이 사용자가 TypeScript를 실제 프로젝트에서 사용해 본 경험이 있는지 확인하기 위함입니다.',
   'example': ["우수 : 'FAVIEW 프로젝트에서 TypeScript를 사용하여 공간 뷰어 랜더링 시 발생할 수 있는 타입 오류를 사전에 방지했습니다. 예를 들어, NaverMap API와의 통합 과정에서 타입 정의를 명확히 하여 데이터 불일치를 예방했습니다.'",
    "보통 : 'TypeScript를 사용하여 타입 오류를 줄였습니다. 주로 변수 선언 시 타입을 명시하여 코드의 안정성을 높였습니다.'",
    "미흡 : 'TypeScript를 사용해 본 적은 있지만, 구체적으로 어떤 문제를 해결했는지는 기억나지 않습니다.'"],
   'reason': '포트폴리오에 TypeScript 사용 경험이 명시되어 있으나, 구체적인 문제 해결 사례가 부족하여 이를 확인하기 위한 질문입니다.'},
  {'question': 'Tanstack Query를 도입하여 프로젝트에서 어떤 성과를 얻었는지 설명해 주실 수 있나요? 특히, 기존의 상태 관리 방식과 비교하여 어떤 점이 개선되었는지 궁금합니다.',
   'tech_keyword': ['tanstack query'],
   'question_type': '경험',
   'purpose': '이 사용자가 Tanstack Query를 실제 프로젝트에서 사용해 본 경험이 있는지 확인하기 위함입니다.',
   'example': ["우수 : 'STORE ADMIN 프로젝트에서 Tanstack Quer

#### **꼬리질문 예시 [JD&PF/교집합]**

In [ ]:
SYSTEM_GENERATE_FOLLOWED_QUESTIONS_CHECKPOINT = """
역할(Role):
* 당신은  다양한 개발 분야에서 폭넓은 경험을 가진 30년차 개발자이며 10년차 IT교육자 김철수입니다.
* 당신은 현재 프론트엔드(FE) 및 백엔드(BE)분야의 기술 역량평가를 진행하는 평가 담당관입니다.
* 역량평가 및 면접을 위해 적절한 질문을 생성해야합니다.

목표(Goal):
* 역량평가 및 면접을 위해 적절한 질문을 생성해야합니다.

* 벡터 DB내 Similarity Search를 통한 결과물인 실제 면접 질문을 참고하여 적절한 면접 질문을 생성합니다.

지시사항(Instructions):
* 각 키워드별로 면접 질문을 키워드마다 각각 3개 이상 생성하세요.
* 검색된 질문이 해당 기술키워드와 관련이 없다면 변형해서 관련있는 질문을 생성하세요.
* 출력의 {tech_keyword}는 Result of Similarity Search[tech_keyword]를 그대로 사용합니다.
* 출력에서 key는 항상 tech_keyword로 사용하세요.

제약사항(Constraints):
* JSON 형식만 반환합니다.
* You must respond only with JSON data without wrapping it in code blocks or any other text.


입력 예시 (Input):
* 형식(Format):
  Result of Similarity Search:
  {
  '기술키워드1': [
    {'original_sentence': '원본 질문 1',
      'searched_question': '원본 질문 1에 대하여 검색된 질문 1',
      'score': '유사도 점수'},
    {'original_sentence': '원본 질문 2',
      'searched_question': '원본 질문 2에 대하여 검색된 질문 2',
      'score': '유사도 점수'},
      ...
    ],
  '기술키워드2': [
    {'original_sentence': '원본 질문 1',
      'searched_question': '원본 질문 1에 대하여 검색된 질문 1',
      'score': '유사도 점수'},
    {'original_sentence': '원본 질문 2',
      'searched_question': '원본 질문 2에 대하여 검색된 질문 2',
      'score': '유사도 점수'},
      ...
    ]
  }
"""

##### **Check-Point / 꼬리질문 / requirements**
변수명 : questions_requirements_in_pf_semantic_search

In [212]:
class SubOutputGenerateFollowedQuestionsCheckpointRequirements(BaseModel):
    tech_keyword: str = Field(description="질문에 해당하는 기술 키워드")
    questions: List[str] = Field(description="생성된 꼬리 질문들")


class OutputGenerateFollowedQuestionsCheckpointRequirements(BaseModel):
    requirements: List[SubOutputGenerateFollowedQuestionsCheckpointRequirements]


USER_GENERATE_FOLLOWED_QUESTIONS_CHECKPOINT_REQUIREMENTS = f"""
Result of Similarity Search: {requirements_in_pf_semantic_search}
"""
followed_questions_checkpoint_requirements = {}
if requirements_in_pf_semantic_search:
    r_followed_questions_checkpoint_requirements = generate_response(
        SYSTEM_GENERATE_FOLLOWED_QUESTIONS_CHECKPOINT,
        USER_GENERATE_FOLLOWED_QUESTIONS_CHECKPOINT_REQUIREMENTS,
        PARAMS_KNOWLEDGE_BASED,
        KWARGS_KNOWLEDGE_BASED,
        "json",
        OutputGenerateFollowedQuestionsCheckpointRequirements,
    )
    cleaned_string = r_followed_questions_checkpoint_requirements.split("{", 1)[
        1
    ].rsplit("}", 1)[0]
    followed_questions_checkpoint_requirements = json.loads("{" + cleaned_string + "}")
else:
    followed_questions_checkpoint_requirements["requirements"] = {
        "tech_keyword": None,
        "questions": None,
    }

In [213]:
followed_questions_checkpoint_requirements

{'requirements': [{'tech_keyword': 'html',
   'questions': ['HTML5에서 추가된 시맨틱 태그의 중요성과 그 사용 사례에 대해 설명해 주세요.',
    'HTML과 CSS를 사용하여 반응형 웹 디자인을 구현하는 방법에 대해 설명해 주세요.',
    'HTML5의 새로운 API 중 하나를 선택하여 설명하고, 그 활용 사례를 들어 주세요.']},
  {'tech_keyword': 'css',
   'questions': ['CSS Flexbox와 Grid 레이아웃의 차이점과 각각의 장단점에 대해 설명해 주세요.',
    'CSS에서 BEM 방법론이 무엇인지 설명하고, 이를 사용하는 이유에 대해 설명해 주세요.',
    'CSS 변수(CSS Variables)를 사용하는 방법과 그 장점에 대해 설명해 주세요.']},
  {'tech_keyword': 'javascript',
   'questions': ['JavaScript의 비동기 처리 방식에 대해 설명하고, Promise와 async/await의 차이점을 설명해 주세요.',
    'JavaScript의 클로저(Closure)가 무엇인지 설명하고, 이를 활용한 예제를 들어 주세요.',
    'JavaScript에서 이벤트 버블링(Event Bubbling)과 캡처링(Event Capturing)의 차이점을 설명해 주세요.']},
  {'tech_keyword': 'typescript',
   'questions': ['TypeScript에서 인터페이스와 타입 별칭(Type Alias)의 차이점에 대해 설명해 주세요.',
    'TypeScript의 제네릭(Generic) 타입을 사용하는 이유와 그 사용 예시에 대해 설명해 주세요.',
    'TypeScript에서 유니온 타입(Union Type)과 인터섹션 타입(Intersection Type)의 차이점을 설명해 주세요.']},
  {'tech_keyword': 'context api',
   'qu

##### **Check-Point / 꼬리질문 / preferences**
변수명 : questions_preferences_in_pf_semantic_search

In [214]:
class SubOutputGenerateFollowedQuestionsCheckpointPreferences(BaseModel):
    tech_keyword: str = Field(description="질문에 해당하는 기술 키워드")
    questions: List[str] = Field(description="생성된 꼬리 질문들")


class OutputGenerateFollowedQuestionsCheckpointPreferences(BaseModel):
    preferences: List[SubOutputGenerateFollowedQuestionsCheckpointPreferences]


USER_GENERATE_FOLLOWED_QUESTIONS_CHECKPOINT_PREFERENCES_JD_AND_PF = f"""
Result of Similarity Search: {preferences_in_pf_semantic_search}
"""
followed_questions_checkpoint_preferences = {}
if preferences_in_pf_semantic_search:
    r_followed_questions_checkpoint_preferences = generate_response(
        SYSTEM_GENERATE_FOLLOWED_QUESTIONS_CHECKPOINT,
        USER_GENERATE_FOLLOWED_QUESTIONS_CHECKPOINT_PREFERENCES_JD_AND_PF,
        PARAMS_KNOWLEDGE_BASED,
        KWARGS_KNOWLEDGE_BASED,
        "json",
        OutputGenerateFollowedQuestionsCheckpointPreferences,
    )
    cleaned_string = r_followed_questions_checkpoint_preferences.split("{", 1)[
        1
    ].rsplit("}", 1)[0]
    followed_questions_checkpoint_preferences = json.loads("{" + cleaned_string + "}")
else:
    followed_questions_checkpoint_preferences["requirements"] = {
        "tech_keyword": None,
        "questions": None,
    }

In [215]:
followed_questions_checkpoint_preferences

{'preferences': [{'tech_keyword': 'ui/ux',
   'questions': ['사용자의 불편함을 분석하고 개선한 경험이 있으시다면, 그 사례를 공유해 주실 수 있을까요?',
    '유저가 더 편리할 것이라고 설득할 때, 추가 기능 개발에 대한 근거로 무엇을 사용하나요?',
    '프로젝트에서 UX 최적화와 관련된 경험이 있으신가요? 있다면, 어떤 부분에서 UX 최적화를 위해 작업했는지 설명해 주세요.',
    'UI 컴포넌트 단위 테스트를 진행할 때 어떤 방법론을 사용하시나요?',
    'Chromatic 배포를 통해 클라이언트 피드백을 반영한 경험이 있으신가요?',
    '반응형 UI 구현 시 가장 중요하게 고려해야 할 요소는 무엇인가요?']}]}

### **Wow-Point**

#### **Extract WowPoint**

In [ ]:
class SubExtractTechWowpoint(BaseModel):
    wowpoint: str = Field(description="기술 관련 wow-point")
    tech_keyword: List[str] = Field(description="기술 키워드")
    reason: str = Field(description="wow-point로 선정한 이유")


class SubExtractExperienceWowpoint(BaseModel):
    wowpoint: str = Field(description="경험 관련 wow-point")
    reason: str = Field(description="wow-point로 선정한 이유")


class OutputExtractWowpoint(BaseModel):
    wowpoint_tech: List[SubExtractTechWowpoint] = Field(
        description="기술 범주의 WOW Point"
    )
    wowpoint_experience: List[SubExtractExperienceWowpoint] = Field(
        description="경험 범주의 WOW Point"
    )


SYSTEM_EXTRACT_WOWPOINT = """
역할(Role):
* 당신은 다양한 개발 분야에서 폭넓은 경험을 가진 30년차 개발자이며, 현재 기술 면접관 입니다.
* 당신은 현재 프론트엔드(FE) 및 백엔드(BE)분야의 기술 면접을 담당해야 합니다.

목표(Goal):
* 주어진 포트폴리오와 기술키워드를 비교해 wow-point를 찾아내야 합니다.

맥락(Context):
* 해당 wow-point는 포트폴리오 내 드러난 강점을 의미합니다.
* 반환 형식은 항상 JSON 형식으로 반환하세요.

지시사항(Instructions):
* wow-point는 기술 범주와 경험 범주로 분류해야 합니다.
* 기술 범주의 wow-point는 반드시 포트폴리오와 기술키워드와 관련있는 내용이되, 포트폴리오 내에 존재하는 문장이어야 합니다.
* 기술 범주에서는 wow-point와 기술 키워드를 함께 반환하세요.

제약사항(Constraints):
* JSON 형식만 반환합니다.
* You must respond only with JSON data without wrapping it in code blocks or any other text.

입력 예시 (Input):
* 형식(Format):
  포트폴리오 : {pf_original},
  기술키워드 : {기술키워드},
"""

USER_EXTRACT_WOWPOINT = f"""
포트폴리오 : {pf_original},
기술키워드 : {keywords_compared_with_keywordlist_and_jd.keys()},
"""
r_wowpoint = generate_response(
    SYSTEM_EXTRACT_WOWPOINT,
    USER_EXTRACT_WOWPOINT,
    PARAMS_KNOWLEDGE_BASED,
    KWARGS_KNOWLEDGE_BASED,
    "json",
    OutputExtractWowpoint,
)

cleaned_string = r_wowpoint.split("{", 1)[1].rsplit("}", 1)[0]
wowpoint = json.loads("{" + cleaned_string + "}")

In [219]:
wowpoint

{'wowpoint_tech': [{'wowpoint': 'Tanstack-Query, React-hook-form 사내 도입',
   'tech_keyword': ['tanstack query'],
   'reason': '포트폴리오에서 Tanstack-Query를 사내에 도입하여 상태 관리의 어려움을 해결한 경험이 기술 키워드와 일치합니다.'},
  {'wowpoint': 'next-auth와 AWS cognito 통합한 로그인 구현',
   'tech_keyword': ['typescript'],
   'reason': '포트폴리오에서 next-auth와 AWS cognito를 통합하여 로그인 구현을 한 경험이 TypeScript와 관련된 기술적 깊이를 보여줍니다.'},
  {'wowpoint': '스토리북을 이용한 소통',
   'tech_keyword': ['ui/ux'],
   'reason': '포트폴리오에서 storybook을 사용하여 UI 컴포넌트 단위 테스트를 진행하고 클라이언트와 소통한 경험이 UI/UX와 관련된 기술적 깊이를 보여줍니다.'}],
 'wowpoint_experience': [{'wowpoint': '3개의 사내 프로덕트를 메인으로 맡아, 섬세한 비즈니스 로직을 구현',
   'reason': '다양한 사내 프로덕트를 주도적으로 개발하며 비즈니스 로직을 구현한 경험은 프로젝트 관리 및 리더십 능력을 보여줍니다.'},
  {'wowpoint': '프론트엔드 스쿨 플러스 단기 심화 과정 최종 발표 심사위원',
   'reason': '프론트엔드 교육 과정에서 심사위원으로 참여한 경험은 교육 및 피드백 능력을 보여줍니다.'}]}

In [ ]:
# 원하는 데이터 구조를 정의합니다.
class OutputGenerateQuestionsWowpoint(BaseModel):
    questions_wowpoint_tech: List[str] = Field(description="지식 관련 질문")
    questions_wowpoint_experience: List[str] = Field(description="경험 범주의 질문")


SYSTEM_GENERATE_QUESTIONS_WOWPOINT = """
역할(Role):
* 당신은  다양한 개발 분야에서 폭넓은 경험을 가진 30년차 개발자이며 10년차 IT교육자 김철수입니다.
* 당신은 현재 프론트엔드(FE) 및 백엔드(BE)분야의 기술 역량평가를 진행하는 평가 담당관입니다.
* 역량평가 및 면접을 위해 적절한 질문을 생성해야합니다.

목표(Goal):
* 역량평가 및 면접을 위해 적절한 질문을 생성해야합니다.

지시사항(Instructions):
* 주어진 문장마다 각각 면접 질문을 생성합니다.
* 생성할 면접 질문은 주어진 WOW Point 기반입니다.
  1. 지식 범주의 질문은 해당 WOW Point에서 질문 가능한 가장 깊은 Depth의 질문을 제시하세요.
  2. 경험 관련 질문은 다음과 같은 Format을 사용하세요.
    Format :
      "[WOW Point]가 정말 인상적인데요. 해당 [WOW Point]를 경험하면서 겪은 문제들 중 가장 힘들었던 것과, 그 문제를 해결한 경험에 대해 소개해주세요."

제약사항(Constraints):
* JSON 형식만 반환합니다.
* You must respond only with JSON data without wrapping it in code blocks or any other text.


입력 예시 (Input):
* 형식(Format):
  WOW-Point:
  {
  'wowpoint_tech': [
    {'wowpoint1': '포트폴리오 내 문장1',
    'tech_keyword': ['tech_keyword A', 'tech_keyword B', ...],
    'reason': 'wowpoint 이유1'},
    {'wowpoint2': '포트폴리오 내 문장2',
    'tech_keyword': ['tech_keyword A', 'tech_keyword B', ...],
    'reason': 'wowpoint 이유2'},
      ...
    ],
 'wowpoint_experience': [
    {'wowpoint1': '포트폴리오 내 문장1',
    'reason': 'wowpoint 이유1'},
    {'wowpoint2': '포트폴리오 내 문장2',
    'reason': 'wowpoint 이유2'},
    ...
    ]
  }
"""

USER_GENERATE_QUESTIONS_WOWPOINT = f"""
WOW-Point: {wowpoint}
"""

r_questions_wowpoint = generate_response(
    SYSTEM_GENERATE_QUESTIONS_WOWPOINT,
    USER_GENERATE_QUESTIONS_WOWPOINT,
    PARAMS_KNOWLEDGE_BASED,
    KWARGS_KNOWLEDGE_BASED,
    "json",
    OutputGenerateQuestionsWowpoint,
)

cleaned_string = r_questions_wowpoint.split("{", 1)[1].rsplit("}", 1)[0]
questions_wowpoint = json.loads("{" + cleaned_string + "}")

In [222]:
questions_wowpoint

{'questions_wowpoint_tech': ['Tanstack-Query를 사내에 도입하여 상태 관리의 어려움을 해결한 경험이 있다고 하셨습니다. Tanstack-Query의 어떤 기능이 기존의 상태 관리 방식보다 더 효과적이었는지 설명해주시고, 도입 과정에서 직면한 가장 큰 기술적 도전은 무엇이었는지 말씀해 주세요.',
  'next-auth와 AWS cognito를 통합하여 로그인 구현을 하셨다고 하셨습니다. 이 과정에서 TypeScript를 활용하여 어떤 문제를 해결하셨는지, 그리고 TypeScript의 어떤 기능이 특히 도움이 되었는지 설명해 주세요.',
  '스토리북을 사용하여 UI 컴포넌트 단위 테스트를 진행하고 클라이언트와 소통하셨다고 하셨습니다. 스토리북을 활용한 UI/UX 개선 과정에서 가장 큰 도전 과제는 무엇이었으며, 이를 어떻게 해결하셨는지 설명해 주세요.'],
 'questions_wowpoint_experience': ['3개의 사내 프로덕트를 메인으로 맡아, 섬세한 비즈니스 로직을 구현하신 경험이 정말 인상적인데요. 해당 경험을 하면서 겪은 문제들 중 가장 힘들었던 것과, 그 문제를 해결한 경험에 대해 소개해주세요.',
  '프론트엔드 스쿨 플러스 단기 심화 과정 최종 발표 심사위원으로 참여하신 경험이 정말 인상적인데요. 해당 경험을 하면서 겪은 문제들 중 가장 힘들었던 것과, 그 문제를 해결한 경험에 대해 소개해주세요.']}

### **Doubt-Point**

In [ ]:
# 원하는 데이터 구조를 정의합니다.
class SubOutputExtractDoubtpointPFOnly(BaseModel):
    original_sentences: str = Field(
        description="포트폴리오 내 의심스러운 정황이 포착된 문장"
    )
    reason: str = Field(description="의심스러운 이유")


# 원하는 데이터 구조를 정의합니다.
class OutputExtractDoubtpointPFOnly(BaseModel):
    doubtsentence_pf_only: list[SubOutputExtractDoubtpointPFOnly]


SYSTEM_EXTRACT_DOUBTPOINT_PF_ONLY = """
역할(Role):
* 당신은 다양한 개발 분야에서 폭넓은 경험을 가진 30년차 개발자이며, 현재 기술 면접관 입니다.
* 당신은 현재 프론트엔드(FE) 및 백엔드(BE)분야의 기술 면접을 담당해야 합니다.

목표(Goal):
* 포트폴리오 자체적으로 의심스러운 정황을 모두 파악해야 합니다.

맥락(Context):
* 의심스러운 정황은 논리적 모순입니다.

지시사항(Instructions):
* 의심스러운 정황은 반드시 포트폴리오에 드러난 내용으로만 판단해야 합니다.
* 반드시 공백 기간이 6개월 이상인 경우만, 공백기에 대한 의심스러운 정황으로 판단합니다.
* 컴퓨터 과학 분야 전공이 아님에도, 졸업 이후 개발 관련 직군 전환이 빠르게 이루어진 경우 의심스러운 정황이 될 수 있습니다.

제약사항(Constraints):
* JSON 형식만 반환합니다.
* You must respond only with JSON data without wrapping it in code blocks or any other text.

입력 예시 (Input):
* 형식(Format):
  포트폴리오 : {"포트폴리오"},

결과 설정(Output):
* 형식(Format):
  class SubOutputExtractDoubtpointPFOnly(BaseModel):
    original_sentences: str = Field(description="포트폴리오 내 의심스러운 정황이 포착된 문장")
    reason: str =  Field(description="의심스러운 이유")
  class OutputExtractDoubtpointPFOnly(BaseModel):
      doubtsentence_pf_only: SubOutputExtractDoubtpointPFOnly = Field(description="포트폴리오 내 문장")
"""

USER_EXTRACT_DOUBTPOINT_PF_ONLY = f"""
포트폴리오 : {pf_original},
"""

r_doubtpoint_pf_only = generate_response(
    SYSTEM_EXTRACT_DOUBTPOINT_PF_ONLY,
    USER_EXTRACT_DOUBTPOINT_PF_ONLY,
    PARAMS_KNOWLEDGE_BASED,
    KWARGS_KNOWLEDGE_BASED,
    "json",
    OutputExtractDoubtpointPFOnly,
)

cleaned_string = r_doubtpoint_pf_only.split("{", 1)[1].rsplit("}", 1)[0]
doubtpoint_pf_only = json.loads("{" + cleaned_string + "}")

In [224]:
doubtpoint_pf_only

{'doubtsentence_pf_only': [{'original_sentences': '졸업 | 스포츠마케팅학과 2017. 03. ~ 2022. 02.',
   'reason': '스포츠마케팅학과 전공임에도 불구하고, 졸업 후 1년 이내에 프론트엔드 개발자로 빠르게 전환한 점이 의심스러울 수 있습니다.'},
  {'original_sentences': '밸류와이즈 개발팀 | 프론트엔드 개발자 | 프리랜서 2023. 11. ~ 2023. 11. (1개월)',
   'reason': '프리랜서로 1개월만 근무한 점이 의심스러울 수 있습니다. 일반적으로 프로젝트의 특성상 짧은 기간이지만, 이력서 상에서의 설명이 부족합니다.'},
  {'original_sentences': 'knccapitalManagement 개발팀 | 프론트엔드 개발자 | 프리랜서 프로젝트 내용: 뉴질랜드 투자 스타트업 회사 소개 페이지 리뉴얼.',
   'reason': '프리랜서로서의 경력이 명확히 기술되어 있지 않으며, 프로젝트의 구체적인 기간이 명시되어 있지 않아 의심스러울 수 있습니다.'}]}

In [ ]:
# 원하는 데이터 구조를 정의합니다.
class OutputGenerateQuestionsDoubtpoint(BaseModel):
    doubt_questions_pf_only: List[str] = Field(
        description="포트폴리오 기반 의심스러운 경향 관련 질문"
    )


SYSTEM_GENERATE_QUESTIONS_DOUBTPOINT = """
역할(Role):
* 당신은  다양한 개발 분야에서 폭넓은 경험을 가진 30년차 개발자이며 10년차 IT교육자 김철수입니다.
* 당신은 현재 프론트엔드(FE) 및 백엔드(BE)분야의 기술 역량평가를 진행하는 평가 담당관입니다.
* 역량평가 및 면접을 위해 적절한 질문을 생성해야합니다.

목표(Goal):
* 주어진 포트폴리오와 의심스러운 정황{doubtPoint}을 비교해 질문을 생성해야합니다.

지시사항(Instructions):
* 주어진 의심스러운 정황에 대해, 포트폴리오에서 의문이 해결된다면 해당 내용에 대한 질문은 생성하지 마세요.
* 질문 생성시, 다음과 같은 Format을 참고하세요. 문장이 자연스럽지 않으면 꼭 Format을 참고하지 않아도 됩니다.
    Format :
      "A라고 이해했는데, B에서 의문점이 생깁니다. A이면 C 아닌가요?
  A는 포트폴리오에서 의심스러운 문장과 관련된 근거, B는 Doubt Point, C는 보편적인 사실을 의미합니다. (C는 자체적으로 판단하되, 구체적인 예시를 제시하세요.)

제약사항(Constraints):
* JSON 형식만 반환합니다.
* You must respond only with JSON data without wrapping it in code blocks or any other text.

입력 예시 (Input):
* 형식(Format):
  doubtpoint_pf_only: {doubtPoint},
  포트폴리오: {pf_original}

결과 설정(Output):
* 형식(Format):
{
  "doubt_questions_pf_only": [
    질문1, 질문2, 질문3, ...
  ],
}
"""

USER_GENERATE_QUESTIONS_DOUBTPOINT = f"""
doubtpoint_pf_only: {doubtpoint_pf_only},
포트폴리오: {pf_original}
"""

r_questions_doubtpoint = generate_response(
    SYSTEM_GENERATE_QUESTIONS_DOUBTPOINT,
    USER_GENERATE_QUESTIONS_DOUBTPOINT,
    PARAMS_KNOWLEDGE_BASED,
    KWARGS_KNOWLEDGE_BASED,
    "json",
    OutputGenerateQuestionsDoubtpoint,
)

cleaned_string = r_questions_doubtpoint.split("{", 1)[1].rsplit("}", 1)[0]
questions_doubtpoint = json.loads("{" + cleaned_string + "}")

In [226]:
questions_doubtpoint

{'doubt_questions_pf_only': ['스포츠마케팅학과를 전공하셨는데, 졸업 후 1년 이내에 프론트엔드 개발자로 전환하신 이유와 그 과정에서 어떤 준비를 하셨는지 궁금합니다.',
  '밸류와이즈 개발팀에서 프리랜서로 1개월만 근무하셨는데, 이 프로젝트의 구체적인 목표와 성과는 무엇이었는지 설명해 주실 수 있나요?',
  'knccapitalManagement 개발팀에서의 프리랜서 프로젝트에 대한 구체적인 기간이 명시되어 있지 않은데, 이 프로젝트는 얼마나 진행되었고, 어떤 성과를 이루셨는지 궁금합니다.']}

# **질문 출력**

In [ ]:
import json

# Pretty print each variable as JSON
print("\n WeakPoint:")
print("##### 대질문 #####")
print("### 자격요건")
print(json.dumps(main_questions_weakpoint_requirements, indent=4, ensure_ascii=False))
print("### 우대사항")
print(json.dumps(main_questions_weakpoint_preferences, indent=4, ensure_ascii=False))
print()
print("##### 꼬리질문 #####")
print("### 자격요건")
print(
    json.dumps(followed_questions_weakpoint_requirements, indent=4, ensure_ascii=False)
)
print("### 우대사항")
print(
    json.dumps(followed_questions_weakpoint_preferences, indent=4, ensure_ascii=False)
)

print("\n CheckPoint :")
print("##### 대질문 #####")
print("### 자격요건")
print(json.dumps(main_questions_checkpoint_requirements, indent=4, ensure_ascii=False))
print("### 우대사항")
print(json.dumps(main_questions_checkpoint_preferences, indent=4, ensure_ascii=False))
print()
print("##### 꼬리질문 #####")
print("### 자격요건")
print(
    json.dumps(followed_questions_checkpoint_requirements, indent=4, ensure_ascii=False)
)
print("### 우대사항")
print(
    json.dumps(followed_questions_checkpoint_preferences, indent=4, ensure_ascii=False)
)

print("------------Wow Point 질문------------")
print(json.dumps(questions_wowpoint, indent=4, ensure_ascii=False))

print("\n ------------의심스러운 정황 질문------------")
print(json.dumps(questions_doubtpoint, indent=4, ensure_ascii=False))


 WeakPoint:
##### 대질문 #####
### 자격요건
{
    "requirements": [
        {
            "question": "채용공고에서 redux를 요구하고 있는데, 포트폴리오/이력서에는 해당 내용이 존재하지 않습니다. 혹시 redux를 사용해보신 경험이나 이론적으로 알고 계신 내용이 있나요?",
            "tech_keyword": [
                "redux"
            ],
            "question_type": "경험",
            "purpose": "이 사용자가 해당 기술을 사용해 봤는지에 대한 사실 확인",
            "example": [
                "우수 : '네, Redux를 사용하여 상태 관리를 구현한 경험이 있습니다. 특히 대규모 애플리케이션에서 상태를 중앙에서 관리하는 데 유용했습니다.'",
                "보통 : 'Redux에 대해 공부한 적은 있지만 실제 프로젝트에 적용해본 적은 없습니다.'",
                "미흡 : 'Redux가 무엇인지 잘 모릅니다.'"
            ],
            "reason": "Redux 사용 경험을 통해 상태 관리에 대한 이해도를 평가하기 위함입니다."
        },
        {
            "question": "채용공고에서 zustand를 요구하고 있는데, 포트폴리오/이력서에는 해당 내용이 존재하지 않습니다. 혹시 zustand를 사용해보신 경험이나 이론적으로 알고 계신 내용이 있나요?",
            "tech_keyword": [
                "zustand"
            ],
            "question_type": "경험",
            "purpose": "이 사용자가 해당 기술을 사용해 봤는지에 대한 사실 확인",
       

# **꼬리 질문**

## **평가 모델**

### **1. LLM을 이용한 질의 확장**
- 원본 질문을 유사 질문들로 증강

In [231]:
def generate_augumented_questions(question):
    system_message = "주어진 면접 질문과 변형하여 면접 질문들을 증강해주세요. 숫자 인덱스 사용하지 말고, 각 질문은 줄바꿈으로 구분하세요"
    default_params = {
        "temperature": 0.0,
        # temperature가 1.0에 가까움 : 더 창의적이고 다양한 결과를 도출
        "max_tokens": 4096,
        # 최대 토큰 수 지정 4o모델의 경우, 4096이 최대
        "top_p": 0.1,
        # 전체 확률 분포의 상위 p%에 포함된 단어를 고려하여 출력
    }
    default_kwargs = {
        "frequency_penalty": 0.5,
        # frequency_penalty가 1.0에 가까움 : 동일한 단어의 반복적인 사용을 크게 억제
        "presence_penalty": 0.0,
        # presence_penalty가 1.0에 가까움 : 생성되는 텍스트에 새로운 주제를 도입하려고 시도함
    }
    full_params = {**default_params, **default_kwargs}

    ### 프롬프트 생성 ###
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_message),
            ("user", question),
        ]
    )
    ###
    llm_expand_query = ChatOpenAI(model="gpt-4o", api_key=OPENAI_API_KEY, **full_params)

    # 출력 파서 설정
    chain = prompt | llm_expand_query | StrOutputParser()
    parsed_response = chain.invoke(
        {"system_input": system_message, "user_input": question}
    )
    return parsed_response

In [ ]:
class OutputGenerateAugumentedQuestions(BaseModel):
    questions: List[str]


def generate_augumented_questions(question):
    system_message = "주어진 면접 질문과 변형하여 면접 질문들을 한글로 증강해주세요. 숫자 인덱스 사용하지 말고, 각 질문은 줄바꿈으로 구분하세요"
    default_params = {
        "temperature": 0.0,
        # temperature가 1.0에 가까움 : 더 창의적이고 다양한 결과를 도출
        "max_tokens": 4096,
        # 최대 토큰 수 지정 4o모델의 경우, 4096이 최대
        "top_p": 0.1,
        # 전체 확률 분포의 상위 p%에 포함된 단어를 고려하여 출력
    }
    default_kwargs = {
        "frequency_penalty": 0.5,
        # frequency_penalty가 1.0에 가까움 : 동일한 단어의 반복적인 사용을 크게 억제
        "presence_penalty": 0.0,
        # presence_penalty가 1.0에 가까움 : 생성되는 텍스트에 새로운 주제를 도입하려고 시도함
    }
    response = generate_response(
        system_message,
        question,
        default_params,
        default_kwargs,
        "json",
        OutputGenerateAugumentedQuestions,
    )
    cleaned_string = response.split("{", 1)[1].rsplit("}", 1)[0]
    parsed_response = json.loads("{" + cleaned_string + "}")
    return parsed_response["questions"]

In [ ]:
test = generate_augumented_questions(
    "Spring에서 `Service`, `Controller`, `Repository` 애노테이션의 차이점은 무엇인가요?"
)
type(test)

dict

In [ ]:
for i in test["questions"]:
    print(i)

type(test["questions"])

Spring에서 `Service` 애노테이션은 어떤 역할을 하나요?
Spring의 `Controller` 애노테이션은 주로 어디에 사용되나요?
`Repository` 애노테이션은 Spring에서 어떤 기능을 제공하나요?
Spring 프레임워크에서 `Service`, `Controller`, `Repository`는 각각 어떤 계층에 속하나요?
Spring의 `Service`, `Controller`, `Repository` 애노테이션을 사용하는 이유는 무엇인가요?
Spring에서 비즈니스 로직을 처리하는 계층에 주로 사용되는 애노테이션은 무엇인가요?
웹 요청을 처리하기 위해 Spring에서 사용하는 애노테이션은 무엇인가요?
데이터베이스와의 상호작용을 위해 Spring에서 사용하는 애노테이션은 무엇인가요?
Spring의 세 가지 주요 계층 구조를 설명하고, 각 계층에 해당하는 애노테이션을 설명해주세요.
Spring 프레임워크에서 각기 다른 역할을 하는 세 가지 주요 컴포넌트는 무엇이며, 이들의 차이점은 무엇인가요?


list

### **2. 검색 및 Top Rank 알고리즘**
- 원본 질문과, 증강된 질문들간의 cosine 유사도 계산

In [248]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


def rank_questions(original_question, augumented_questions):

    vectorizer = TfidfVectorizer().fit_transform(
        [original_question] + augumented_questions
    )
    vectors = vectorizer.toarray()
    input_vector = vectors[0]
    candidate_vectors = vectors[1:]

    # 코사인 유사도 계산
    similarities = cosine_similarity([input_vector], candidate_vectors)[0]
    ranked_indices = np.argsort(similarities)[::-1]
    ranked_questions = [
        (augumented_questions[i], similarities[i]) for i in ranked_indices
    ]
    return ranked_questions

### **3. 증강된 질문에 대한 답변 생성**
- 증강된 질문들 중, 원본 질문과의 유사도 가장 높은 3개 질문들에 대한 답변 생성

In [ ]:
class OutputGenerateAnswers(BaseModel):
    question: str = Field(description="생성된 질문")
    best_answer: str = Field(description="모범답안")
    tech_keyword: str = Field(description="답변 생성시 추출한 기술키워드")
    link: str = Field(description="모범답안 생성시 사용한 기술키워드의 공식문서 링크")
    example: List[str] = Field(
        description="우수 : '우수 응답 가이드', 보통 : '보통 응답 가이드', 미흡 : '미흡 응답 가이드'"
    )


def generate_answers(question):
    system_message = """
    다음 주어진 3개의 면접 질문들에 대해, 각각 1~2문장의 모범답안을 제시해주세요.
    답변 생성시 반드시 질문의 기술키워드를 추출하고, 추출된 기술키워드의 공식문서를 참고하여 답변을 생성하세요.
    또한, 주어진 질문에 대해 우수, 보통, 미흡 응답 가이드를 작성하세요.
    응답 가이드는 사용자의 답변을 평가하기 위한 용도로 사용될 예정입니다.
    * json 형식으로 출력하세요.
    """
    response = generate_response(
        system_message,
        question,
        PARAMS_KNOWLEDGE_BASED,
        KWARGS_KNOWLEDGE_BASED,
        "json",
        OutputGenerateAnswers,
    )
    cleaned_string = response.split("{", 1)[1].rsplit("}", 1)[0]
    answer = json.loads("{" + cleaned_string + "}")

    # response = generate_response(system_message, questions, "list")
    # print(f"그냥 응답 {response}")
    # # 불필요한 부분 제거 및 JSON 파싱
    # cleaned_string = response.split("{", 1)[1].rsplit("}", 1)[0]
    # QnA = json.loads("{" + cleaned_string + "}")
    # print(f"json형식 변환 :{QnA}")
    return answer

In [260]:
test = generate_answers("Spring에서 `Service` 애노테이션은 어떤 역할을 하나요?")
test

{'question': 'Spring에서 `Service` 애노테이션은 어떤 역할을 하나요?',
 'best_answer': '`@Service` 애노테이션은 Spring에서 비즈니스 로직을 수행하는 서비스 계층을 나타내며, 주로 서비스 클래스에 사용되어 해당 클래스가 서비스 역할을 한다는 것을 명시합니다.',
 'tech_keyword': '@Service annotation',
 'link': 'https://docs.spring.io/spring-framework/docs/current/javadoc-api/org/springframework/stereotype/Service.html',
 'example': ["우수 : '@Service' 애노테이션은 비즈니스 로직을 처리하는 서비스 계층을 나타내며, 스프링 컨테이너가 해당 클래스를 서비스로 인식하도록 합니다.",
  "보통 : '@Service'는 서비스 클래스에 붙이는 애노테이션입니다.",
  "미흡 : '@Service'는 스프링에서 사용되는 애노테이션입니다."]}

### **4. 평가점수 산출**

In [283]:
# 원하는 데이터 구조를 정의합니다.
class OutputEvaluateQuestion(BaseModel):
    score: int = Field(description="0~5점 사이의 평가 점수")
    reason: str = Field(description="평가 이유")
    tech_keywords: List[str] = Field(description="부족한 키워드 리스트")


def evaluate_answer(original_question, original_answer, QnA):
    system_message = """
      당신은 FE, BE 면접을 담당하는 면접관입니다.
      주어진 면집 질문에 대해서, 0~5점 사이의 평가 점수와 평가 이유를 제시하세요.

      * 부족한 기술 키워드는 입력된 '사전 기술 키워드'에서 찾아서 사용하세요.
      * '사용자 답변'을 '참고 데이터'와 비교하여 '사용자 답변'을 평가하세요.
      * 평가 척도는 입력의 '응답 가이드'를 참고하세요.
      * 평가 척도는 다음과 같습니다.
        0점 : 대답이 아예 다른 내용이거나 모른다고 대답함.
        1점 : 대답이 불충분하거나 주제에서 벗어남.
        2점 : 미흡에 해당하는 답변 제공.
        3점 : 보통에 해당하는 답변으로 대부분의 기본 내용을 포함하나, 구체적인 예시나 세부 사항이 부족함.
        4점 : 구체적인 예시나 세부사항을 포함했으나, 그 답변의 질이 부족함.
        5점 : 전문적 지식 및 구체적인 경험을 바탕으로 깊이 있는 답변 제공.
      * json 형식으로 출력하세요.
    """
    user_message = f"질문 : {original_question}, 사용자 답변 : {original_answer}, 참고 데이터 : {QnA}, 사전 기술 키워드 : {unique_keywords}"
    # r_score = generate_response(system_message, user_message)

    # cleaned_string = r_score.split("{", 1)[1].rsplit("}", 1)[0]
    # score = json.loads("{" + cleaned_string + "}")

    response = generate_response(
        system_message,
        user_message,
        PARAMS_KNOWLEDGE_BASED,
        KWARGS_KNOWLEDGE_BASED,
        "json",
        OutputEvaluateQuestion,
    )
    cleaned_string = response.split("{", 1)[1].rsplit("}", 1)[0]
    evaluation = json.loads("{" + cleaned_string + "}")
    return evaluation

In [284]:
s_test = evaluate_answer(
    "Spring에서 `Service`, `Controller`, `Repository` 애노테이션의 차이점은 무엇인가요?",
    "처음 지금 그거 도입했을 때는 말씀드린 어드민에서 적용을 해봤고요. 기존에 있던 것은 따로 상태 관리에 대해서 이야기 나눌 시간이 이야기 나눌 만큼 조금 여유로운 스프린트가 아니었다 보니까 컨텍스트 API로 일단은 모든 걸 관리하고 있었거든요. 근데 지금 말씀드린 통합한 홈페이지와 화교라는 공간 랜더링 서비스를 통합한 곳에서 통합한 프로젝트가 CRA로 이루어져 있는데 지금 제가 Nest JS로 마이그레이션을 지금 하고 있는 상황이거든요. 그래서 거기에서 지금 Tanstack 쿼리로 하자고 제의를 드렸어요. 그 이유는 컨텍스트 API는 해당 감싸져 있는 모든 children들이 리렌더링이 일어나면 모두 다 되잖아요. 근데 제가 알기로는 Tanstack 쿼리는 리렌더링이 되긴 하는데 해당 데이터 쿼리가 선언된 데이터와 관련된 컴포넌트만 리렌더링이 된다고 알고 있어서 유저 플랜 이런 전형적으로 쓰는 것들은 계속 리렌더링이 일어날 수 있으니 Tanstack 쿼리로 마이그레이션 해서 조금 더 리렌더링을 최소화하자라고 이렇게 같이 이야기가 잘 소통이 잘 되어서 실제로 말씀하신 대로 Tanstck 쿼리로 마이그레이션을 하고 있습니다.조금 힘들긴 하지만 하고 있습니다.",
    test,
)

In [285]:
type(s_test["score"])

int

In [286]:
def evaluation_question(input_question, input_answer):
    ### 1. LLM을 이용한 질의 확장 ###
    expanded_questions = generate_augumented_questions(input_question)

    print(f"expanded_questions:{expanded_questions}")

    ### 2. 검색 및 Top Rank 알고리즘 ###
    ranked_questions = rank_questions(input_question, expanded_questions)

    print(f"ranked_questions:{ranked_questions}")

    ### 3. 증강된 질문에 대한 답변 생성 ###
    augumented_datas = []
    for generated_question in [q[0] for q in ranked_questions[:3]]:
        augumented_datas.append(
            generate_answers(generated_question)
        )  # 상위 3개 질문에 대해 답변 생성

    print(f"augumented_datas: {augumented_datas}")
    print("-" * 200)

    ### 4. 답변 리스트 작성 및 Top Rank 재정렬 ###
    # ranked_answers = rank_answers(input_question, generated_answers)

    # prompt_answer = []
    # for answer in ranked_answers:
    #   prompt_answer.append(answer[1])

    ### 4. 평가점수 산출 ###
    evaluation = evaluate_answer(input_question, input_answer, augumented_datas)

    # 결과 출력
    print(f"점수 : {evaluation['score']}")
    print(f"평가 이유 : {evaluation['reason']}")
    print(f"부족한 기술 키워드 : {evaluation['tech_keywords']}")
    print("-" * 200)

    return evaluation

## **꼬리질문 생성 모델**

In [ ]:
# 원하는 데이터 구조를 정의합니다.
class OutputFollowedQuestion(BaseModel):
    question: str


SYSTEM_FOLLOWED_QUESTION = """
역할(Role):
* 당신은  다양한 개발 분야에서 폭넓은 경험을 가진 30년차 개발자이며 10년차 IT교육자 김철수입니다.
* 당신은 현재 프론트엔드(FE) 및 백엔드(BE)분야의 기술 역량평가를 진행하는 평가 담당관입니다.
* 역량평가 및 면접을 위해 적절한 질문을 생성해야합니다.

목표(Goal):
* 역량평가 및 면접을 위해 적절한 기술 꼬리 질문을 생성해야합니다.
* 벡터 DB내 Similarity Search를 통한 결과물인 실제 면접 질문을 참고하여 적절한 기술 면접 질문을 생성합니다.

지시사항(Instructions):
* 입력된 'history'와 'Result of Similarity Search'를 참고하여, 꼬리질문을 하나 생성하세요.
* 출력의 {tech_keyword}는 Result of Similarity Search[tech_keyword]를 그대로 사용합니다.
* 출력에서 key는 항상 tech_keyword로 사용하세요.

제약사항(Constraints):
* JSON 형식만 반환합니다.
* You must respond only with JSON data without wrapping it in code blocks or any other text.


입력 예시 (Input):
* 형식(Format):
  history: {history},
  Result of Similarity Search:
  {
  '기술키워드1': [
    {'original_sentence': '원본 질문 1',
      'searched_question': '원본 질문 1에 대하여 검색된 질문 1',
      'score': '유사도 점수'},
    {'original_sentence': '원본 질문 2',
      'searched_question': '원본 질문 2에 대하여 검색된 질문 2',
      'score': '유사도 점수'},
      ...
    ],
  '기술키워드2': [
    {'original_sentence': '원본 질문 1',
      'searched_question': '원본 질문 1에 대하여 검색된 질문 1',
      'score': '유사도 점수'},
    {'original_sentence': '원본 질문 2',
      'searched_question': '원본 질문 2에 대하여 검색된 질문 2',
      'score': '유사도 점수'},
      ...
    ]
  }
"""


def followed_question(q, history, namespace):
    followed_questions = []
    searched_datasets = []
    for tech_keyword in q["tech_keywords"]:
        searched_data = search_vector_db(tech_keyword, tech_keyword, namespace)
        for m in searched_data["matches"]:
            searched_datasets.append(m["metadata"]["text"])

    USER_FOLLOWED_QUESTION = f"""
    history: {history},
    Result of Similarity Search : {searched_datasets}
    """
    r_followed_questions = generate_response(
        SYSTEM_FOLLOWED_QUESTION,
        USER_FOLLOWED_QUESTION,
        PARAMS_KNOWLEDGE_BASED,
        KWARGS_KNOWLEDGE_BASED,
        "json",
        OutputFollowedQuestion,
    )
    # 불필요한 부분 제거 및 JSON 파싱
    cleaned_string = r_followed_questions.split("{", 1)[1].rsplit("}", 1)[0]
    json_data = json.loads("{" + cleaned_string + "}")
    followed_questions.append(json_data["question"])
    return searched_datasets, followed_questions

**Test Data**

In [288]:
tech_field = "BE"

input_question = """
Spring에서 `Service`, `Controller`, `Repository` 애노테이션의 차이점은 무엇인가요?
"""

input_answer = """
처음 지금 그거 도입했을 때는 말씀드린 어드민에서 적용을 해봤고요. 기존에 있던 것은 따로 상태 관리에 대해서 이야기 나눌 시간이 이야기 나눌 만큼 조금 여유로운 스프린트가 아니었다 보니까 컨텍스트 API로 일단은 모든 걸 관리하고 있었거든요. 근데 지금 말씀드린 통합한 홈페이지와 화교라는 공간 랜더링 서비스를 통합한 곳에서 통합한 프로젝트가 CRA로 이루어져 있는데 지금 제가 Nest JS로 마이그레이션을 지금 하고 있는 상황이거든요. 그래서 거기에서 지금 Tanstack 쿼리로 하자고 제의를 드렸어요. 그 이유는 컨텍스트 API는 해당 감싸져 있는 모든 children들이 리렌더링이 일어나면 모두 다 되잖아요. 근데 제가 알기로는 Tanstack 쿼리는 리렌더링이 되긴 하는데 해당 데이터 쿼리가 선언된 데이터와 관련된 컴포넌트만 리렌더링이 된다고 알고 있어서 유저 플랜 이런 전형적으로 쓰는 것들은 계속 리렌더링이 일어날 수 있으니 Tanstack 쿼리로 마이그레이션 해서 조금 더 리렌더링을 최소화하자라고 이렇게 같이 이야기가 잘 소통이 잘 되어서 실제로 말씀하신 대로 Tanstck 쿼리로 마이그레이션을 하고 있습니다.조금 힘들긴 하지만 하고 있습니다.
"""

In [293]:
namespace = tech_field
count = 0
score = 0
history = []
q = input_question

while True:
    print(f"질문 : {q}")
    answer = input("답변 : ")
    print("답변 : {answer}")
    print("-" * 200)
    count += 1  # 3-depth 까지만 꼬리질문 생성
    evaluation = evaluation_question(q, answer)
    score += int(evaluation["score"])
    history.append({"질문": q, "답변": answer, "평가": evaluation})
    ### --- 테스트용 출력 --- ###
    # h = history[-1]
    # print("-"*25, count ,"-"*25)
    # print(f"질문 : {h['질문']}")
    # print(f"답변 : {h['답변']}")
    # print(f"평가 : {h['평가']}")
    # print("-"*50)

    if count >= 3 or int(evaluation["score"]) == 0:
        print(f"<테스트>의 평가점수는 {round((score / count),2)}입니다.")
        break

    searched_question, generated_question = followed_question(
        evaluation, history, namespace
    )
    print(f"searched_question : {searched_question}")
    print(f"generated_question : {generated_question}")
    q = generated_question[0]

질문 : 
Spring에서 `Service`, `Controller`, `Repository` 애노테이션의 차이점은 무엇인가요?

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
expanded_questions:['Spring에서 `Service` 애노테이션은 어떤 역할을 하나요?', 'Spring의 `Controller` 애노테이션은 주로 어디에 사용되나요?', 'Spring에서 `Repository` 애노테이션은 어떤 기능을 제공하나요?', '`Service`, `Controller`, `Repository` 애노테이션의 공통점과 차이점은 무엇인가요?', 'Spring 프레임워크에서 각 계층별로 사용하는 주요 애노테이션에 대해 설명해 주세요.', 'Spring에서 비즈니스 로직을 처리하는 계층에 주로 사용되는 애노테이션은 무엇인가요?', '데이터 접근 계층에서 사용하는 Spring 애노테이션은 무엇이며, 그 역할은 무엇인가요?', '웹 요청을 처리하기 위해 Spring에서 사용하는 주요 애노테이션은 무엇인가요?', 'Spring의 각 계층별로 적절한 애노테이션을 선택하는 기준이 있나요? 있다면 설명해 주세요.', 'Spring 프레임워크에서 MVC 패턴 구현 시 각 컴포넌트에 맞는 애노테이션 사용법을 설명해 주세요.']
ranked_questions:[('`Service`, `Controller`, `Repository` 애노테이션의 공통점과 차이점은 무엇인가요?', np.float64(0.8398544685624718)), ('Spring에서 `Service` 애노테이션은 어떤 역할을 하나요?', np.float64(0.237403564059445

In [ ]:
a = """ 사용자는 useEffect 훅이 React의 클래스 컴포넌트 라이프사이클 메서드인 componentDidMount, componentDidUpdate, componentWillUnmount와 어떻게 연관되어 있는지를 잘 설명했습니다. 의존성 배열을 통한 업데이트 제어와 리턴 콜백에 대한 언급도 포함되어 있습니다. 그러나 전체적인 설명에서 useEffect의 장점이나 구체적인 예시가 부족합니다."""

In [ ]:
temp1 = """{
  "question": "Spring 프레임워크에서 `Service`, `Controller`, `Repository` 애노테이션의 역할과 차이점은 무엇인가요?",
  "best_answer": "`@Service`는 비즈니스 로직을 처리하는 컴포넌트를 나타내고, `@Controller`는 웹 요청을 처리하여 응답을 반환하는 데 사용됩니다. `@Repository`는 데이터 액세스 계층을 담당하며 데이터베이스와의 상호작용을 처리합니다.",
  "tech_keyword": ["Spring Framework", "@Service", "@Controller", "@Repository"],
  "link": "https://docs.spring.io/spring-framework/docs/current/reference/html/web.html#mvc-controller",
  "answer_guide": {
    "우수": "각 애노테이션의 목적과 역할에 대해 정확히 설명하고, 세부적인 차이점까지 언급할 수 있습니다.",
    "보통": "애노테이션의 기본 역할에 대한 이해를 보여주지만, 구체적인 차이점을 명확히 설명하지 못합니다.",
    "미흡": "애노테이션들의 기본적인 역할조차 제대로 설명하지 못하거나 혼동된 설명을 제공합니다."
  }
}
"""

cleaned_temp1 = temp1.split("{", 1)[1].rsplit("}", 1)[0]

In [ ]:
cleaned_temp1

'\n  "question": "Spring 프레임워크에서 `Service`, `Controller`, `Repository` 애노테이션의 역할과 차이점은 무엇인가요?",\n  "best_answer": "`@Service`는 비즈니스 로직을 처리하는 컴포넌트를 나타내고, `@Controller`는 웹 요청을 처리하여 응답을 반환하는 데 사용됩니다. `@Repository`는 데이터 액세스 계층을 담당하며 데이터베이스와의 상호작용을 처리합니다.",\n  "tech_keyword": ["Spring Framework", "@Service", "@Controller", "@Repository"],\n  "link": "https://docs.spring.io/spring-framework/docs/current/reference/html/web.html#mvc-controller",\n  "answer_guide": {\n    "우수": "각 애노테이션의 목적과 역할에 대해 정확히 설명하고, 세부적인 차이점까지 언급할 수 있습니다.",\n    "보통": "애노테이션의 기본 역할에 대한 이해를 보여주지만, 구체적인 차이점을 명확히 설명하지 못합니다.",\n    "미흡": "애노테이션들의 기본적인 역할조차 제대로 설명하지 못하거나 혼동된 설명을 제공합니다."\n  }\n'

In [ ]:
temp2 = """```json
{
  "question": "`Service`, `Controller`, 및 `Repository` 애노테이션은 Spring에서 어떻게 사용되며, 이들 사이의 주요 차이점은 무엇인가요?",
  "best_answer": "`@Service`는 비즈니스 로직을 처리하는 클래스에 사용되며, `@Controller`는 HTTP 요청을 처리하고 응답을 반환하는 데 사용됩니다. `@Repository`는 데이터 액세스 계층에서 데이터베이스와의 상호 작용을 담당하는 클래스에 적용됩니다.",
  "tech_keyword": ["@Service", "@Controller", "@Repository"],
  "link": "https://docs.spring.io/spring-framework/docs/current/reference/html/core.html#beans-stereotype-annotations",
  "answer_guide": {
    "우수": "`@Service`, `@Controller`, `@Repository` 각각의 역할과 책임을 명확하게 설명하며, 이들 간의 차이점을 정확히 이해하고 있음을 보여줍니다.",
    "보통": "`@Service`, `@Controller`, 및 `@Repository`의 기본적인 역할만 설명하지만, 세부적인 차이점이나 그 사용 사례에 대한 설명은 부족합니다.",
    "미흡": 애노테이션들의 역할이나 목적에 대한 이해가 부족하여 잘못된 정보나 불명확한 답변을 제공합니다."
  }
}
"""

cleaned_temp2 = temp1.split("{", 1)[1].rsplit("}", 1)[0]

In [ ]:
cleaned_temp2

'\n  "question": "Spring 프레임워크에서 `Service`, `Controller`, `Repository` 애노테이션의 역할과 차이점은 무엇인가요?",\n  "best_answer": "`@Service`는 비즈니스 로직을 처리하는 컴포넌트를 나타내고, `@Controller`는 웹 요청을 처리하여 응답을 반환하는 데 사용됩니다. `@Repository`는 데이터 액세스 계층을 담당하며 데이터베이스와의 상호작용을 처리합니다.",\n  "tech_keyword": ["Spring Framework", "@Service", "@Controller", "@Repository"],\n  "link": "https://docs.spring.io/spring-framework/docs/current/reference/html/web.html#mvc-controller",\n  "answer_guide": {\n    "우수": "각 애노테이션의 목적과 역할에 대해 정확히 설명하고, 세부적인 차이점까지 언급할 수 있습니다.",\n    "보통": "애노테이션의 기본 역할에 대한 이해를 보여주지만, 구체적인 차이점을 명확히 설명하지 못합니다.",\n    "미흡": "애노테이션들의 기본적인 역할조차 제대로 설명하지 못하거나 혼동된 설명을 제공합니다."\n  }\n'

In [ ]:

data = {
    "Question": "Redux에서 액션(action)과 리듀서(reducer)의 역할과 이들이 상태 관리에 어떻게 기여하는지 설명해 주실 수 있나요?",
    "User Answer": "Redux에서 액션은 상태 변화를 설명하는 객체이고, 리듀서는 현재 상태와 액션을 기반으로 새로운 상태를 반환하는 순수 함수로, 둘은 함께 상태 변경 과정을 예측 가능하고 일관성 있게 만듭니다.",
    "Model Answers": [
        "리덕스에서 액션 크리에이터는 특정 액션 객체를 생성하는 함수이며, 리듀서는 이 액션 객체를 받아 상태를 업데이트하는 함수입니다. 즉, 액션 크리에이터가 생성한 액션을 리듀서가 처리하여 상태를 변경합니다.",
        "Redux에서 액션은 상태 변화를 설명하는 객체로, type 필드를 필수로 포함하며, 리듀서는 액션에 따라 상태를 변경하는 순수 함수로 설계해야 합니다. 액션은 명확하고 구체적인 type을 가지도록 하고, 리듀서는 불변성을 유지하며 상태를 업데이트하도록 설계합니다.",
        "Redux에서 상태를 업데이트하기 위해, 먼저 액션 객체를 생성하여 상태 변경의 의도를 설명합니다. 그런 다음, 리듀서는 현재 상태와 액션을 받아 새로운 상태를 반환하는 순수 함수로, 상태 업데이트를 수행합니다."
    ],
    "Score": 4,
    "Evaluation": "사용자 답변은 Redux에서 액션과 리듀서의 기본 역할을 잘 설명하고 있으며, 상태 관리에 어떻게 기여하는지에 대한 이해도도 잘 드러내고 있습니다. 액션이 상태 변화를 설명하는 객체라는 점과 리듀서가 현재 상태와 액션을 기반으로 새로운 상태를 반환하는 순수 함수라는 점을 명확히 설명했습니다. 그러나 구체적인 예시나 세부 사항이 부족하여 5점에 미치지 못합니다. 예를 들어, 액션 객체의 구조나 리듀서의 불변성 유지에 대한 언급이 있었다면 더 높은 점수를 받을 수 있었을 것입니다."
}

# Define a function to format long text by splitting at periods for better readability
def format_long_text(text):
    sentences = text.split(". ")
    formatted_text = "\n".join([sentence + "." for sentence in sentences if sentence])
    return formatted_text

# Print the Redux analysis with formatted evaluation
def print_redux_analysis_with_format(data):
    print("=== Redux Interview Question Analysis ===\n")
    print(f"Question:\n{data['Question']}\n")
    print("User Answer:")
    print(f"{data['User Answer']}\n")
    print("Model Answers:")
    for i, answer in enumerate(data['Model Answers'], start=1):
        print(f"  Model Answer {i}:\n  {answer}\n")
    print(f"Score: {data['Score']}\n")
    print("Evaluation:")
    print(format_long_text(data['Evaluation']))
    print()

print_redux_analysis_with_format(data)

=== Redux Interview Question Analysis ===

Question:
Redux에서 액션(action)과 리듀서(reducer)의 역할과 이들이 상태 관리에 어떻게 기여하는지 설명해 주실 수 있나요?

User Answer:
Redux에서 액션은 상태 변화를 설명하는 객체이고, 리듀서는 현재 상태와 액션을 기반으로 새로운 상태를 반환하는 순수 함수로, 둘은 함께 상태 변경 과정을 예측 가능하고 일관성 있게 만듭니다.

Model Answers:
  Model Answer 1:
  리덕스에서 액션 크리에이터는 특정 액션 객체를 생성하는 함수이며, 리듀서는 이 액션 객체를 받아 상태를 업데이트하는 함수입니다. 즉, 액션 크리에이터가 생성한 액션을 리듀서가 처리하여 상태를 변경합니다.

  Model Answer 2:
  Redux에서 액션은 상태 변화를 설명하는 객체로, type 필드를 필수로 포함하며, 리듀서는 액션에 따라 상태를 변경하는 순수 함수로 설계해야 합니다. 액션은 명확하고 구체적인 type을 가지도록 하고, 리듀서는 불변성을 유지하며 상태를 업데이트하도록 설계합니다.

  Model Answer 3:
  Redux에서 상태를 업데이트하기 위해, 먼저 액션 객체를 생성하여 상태 변경의 의도를 설명합니다. 그런 다음, 리듀서는 현재 상태와 액션을 받아 새로운 상태를 반환하는 순수 함수로, 상태 업데이트를 수행합니다.

Score: 4

Evaluation:
사용자 답변은 Redux에서 액션과 리듀서의 기본 역할을 잘 설명하고 있으며, 상태 관리에 어떻게 기여하는지에 대한 이해도도 잘 드러내고 있습니다.
액션이 상태 변화를 설명하는 객체라는 점과 리듀서가 현재 상태와 액션을 기반으로 새로운 상태를 반환하는 순수 함수라는 점을 명확히 설명했습니다.
그러나 구체적인 예시나 세부 사항이 부족하여 5점에 미치지 못합니다.
예를 들어, 액션 객체의 구조나 리듀서의 불변성 유지에 대한 언급이 있었다면 더 높은 점수를 받을 수 있었을 것입니다